# ROW_NUMBER(), RANK(), DENSE_RANK() - Complete Deep Dive

## The Core Concept: What Are These Functions?

These are **window functions** that assign numbers to rows based on some ordering criteria. Think of them as different ways to "number" your data rows, each with specific rules about how they handle ties (duplicate values).

### The Mental Model
Imagine you're ranking students in a class by their test scores:
- **ROW_NUMBER()**: Every student gets a unique number, even if they have the same score
- **RANK()**: Students with the same score get the same rank, but we skip numbers afterward
- **DENSE_RANK()**: Students with the same score get the same rank, but we DON'T skip numbers

## Function Syntax & Behavior

```sql
ROW_NUMBER() OVER (PARTITION BY column ORDER BY column)
RANK() OVER (PARTITION BY column ORDER BY column) 
DENSE_RANK() OVER (PARTITION BY column ORDER BY column)
```

### Key Components:
- **OVER()**: Defines the window (scope of rows to consider)
- **PARTITION BY**: Divides data into groups (optional)
- **ORDER BY**: Defines the ranking criteria (required)

## Detailed Breakdown

### ROW_NUMBER()
**What it does**: Assigns a unique sequential integer to each row
**Ties handling**: No ties - every row gets a different number
**Use when**: You need unique identifiers or want to pick exactly N rows

### RANK()
**What it does**: Assigns the same rank to rows with identical values
**Ties handling**: Gaps in sequence (1, 2, 2, 4, 5...)
**Use when**: Traditional ranking like sports leaderboards

### DENSE_RANK()
**What it does**: Same rank for identical values, but no gaps
**Ties handling**: No gaps in sequence (1, 2, 2, 3, 4...)
**Use when**: You want consecutive ranking numbers

## Real-World Examples

### Example 1: Employee Salary Ranking
```sql
-- Sample data: employees table
CREATE TABLE employees (
    emp_id INT,
    name VARCHAR(50),
    department VARCHAR(50),
    salary DECIMAL(10,2)
);

-- Ranking queries
SELECT 
    name,
    department,
    salary,
    ROW_NUMBER() OVER (ORDER BY salary DESC) as row_num,
    RANK() OVER (ORDER BY salary DESC) as rank_pos,
    DENSE_RANK() OVER (ORDER BY salary DESC) as dense_rank_pos
FROM employees;
```

**Result with sample data:**
| Name | Department | Salary | ROW_NUMBER | RANK | DENSE_RANK |
|------|------------|--------|------------|------|------------|
| Alice | Engineering | 95000 | 1 | 1 | 1 |
| Bob | Engineering | 90000 | 2 | 2 | 2 |
| Carol | Marketing | 90000 | 3 | 2 | 2 |
| David | Sales | 85000 | 4 | 4 | 3 |

### Example 2: Top N Per Group
```sql
-- Get top 3 highest paid employees per department
WITH ranked_employees AS (
    SELECT 
        name,
        department,
        salary,
        ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary DESC) as rn
    FROM employees
)
SELECT * FROM ranked_employees WHERE rn <= 3;
```

## Popular Use Cases

### 1. **Pagination & Data Sampling**
```sql
-- Get rows 11-20 for pagination
WITH numbered_rows AS (
    SELECT *, ROW_NUMBER() OVER (ORDER BY created_date) as rn
    FROM products
)
SELECT * FROM numbered_rows WHERE rn BETWEEN 11 AND 20;
```

### 2. **Removing Duplicates**
```sql
-- Remove duplicate customers, keeping the latest record
WITH dedupe AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY updated_date DESC) as rn
    FROM customers
)
SELECT * FROM dedupe WHERE rn = 1;
```

### 3. **Finding Top/Bottom N**
```sql
-- Top 5 products by revenue in each category
WITH product_ranks AS (
    SELECT 
        product_name,
        category,
        revenue,
        DENSE_RANK() OVER (PARTITION BY category ORDER BY revenue DESC) as rank_pos
    FROM product_sales
)
SELECT * FROM product_ranks WHERE rank_pos <= 5;
```

### 4. **Percentile Analysis**
```sql
-- Divide customers into quartiles by spending
SELECT 
    customer_id,
    total_spent,
    NTILE(4) OVER (ORDER BY total_spent) as quartile,
    DENSE_RANK() OVER (ORDER BY total_spent DESC) as spending_rank
FROM customer_spending;
```

## Python Pandas Equivalents

### ROW_NUMBER()
```python
# SQL: ROW_NUMBER() OVER (ORDER BY salary DESC)
df['row_num'] = df.sort_values('salary', ascending=False).reset_index(drop=True).index + 1

# With grouping
df['row_num'] = df.sort_values('salary', ascending=False).groupby('department').cumcount() + 1
```

### RANK()
```python
# SQL: RANK() OVER (ORDER BY salary DESC)
df['rank'] = df['salary'].rank(method='min', ascending=False)

# With grouping
df['rank'] = df.groupby('department')['salary'].rank(method='min', ascending=False)
```

### DENSE_RANK()
```python
# SQL: DENSE_RANK() OVER (ORDER BY salary DESC)
df['dense_rank'] = df['salary'].rank(method='dense', ascending=False)

# With grouping
df['dense_rank'] = df.groupby('department')['salary'].rank(method='dense', ascending=False)
```

## Excel Equivalents

### ROW_NUMBER()
```excel
=ROW() - 1  # Simple sequential numbering
=COUNTIF($A$1:A1, A1)  # For grouped numbering
```

### RANK()
```excel
=RANK(B2, $B$2:$B$10, 0)  # 0 for descending order
```

### DENSE_RANK()
Excel doesn't have a direct equivalent, but you can use:
```excel
=RANK(B2, $B$2:$B$10, 0) - COUNTIF($B$2:B1, ">"&B2)
```

## Performance Considerations

### 1. **Index Strategy**
- Create indexes on columns used in ORDER BY
- Composite indexes for PARTITION BY + ORDER BY

### 2. **Memory Usage**
- Window functions can be memory-intensive
- Consider limiting result sets with WHERE clauses

### 3. **Query Optimization**
```sql
-- GOOD: Filter first, then rank
WITH filtered_data AS (
    SELECT * FROM large_table WHERE date >= '2024-01-01'
)
SELECT *, ROW_NUMBER() OVER (ORDER BY amount DESC) as rn
FROM filtered_data;

-- AVOID: Ranking all data then filtering
SELECT * FROM (
    SELECT *, ROW_NUMBER() OVER (ORDER BY amount DESC) as rn
    FROM large_table
) WHERE date >= '2024-01-01';
```

## Common Interview Questions

### Q1: "Get the 2nd highest salary in each department"
```sql
WITH salary_ranks AS (
    SELECT 
        emp_id, 
        department, 
        salary,
        DENSE_RANK() OVER (PARTITION BY department ORDER BY salary DESC) as rank_pos
    FROM employees
)
SELECT * FROM salary_ranks WHERE rank_pos = 2;
```

### Q2: "Find duplicate records"
```sql
WITH duplicate_check AS (
    SELECT 
        *,
        ROW_NUMBER() OVER (PARTITION BY email ORDER BY created_date) as rn
    FROM users
)
SELECT * FROM duplicate_check WHERE rn > 1;
```

### Q3: "Calculate running totals with rankings"
```sql
SELECT 
    order_date,
    amount,
    SUM(amount) OVER (ORDER BY order_date) as running_total,
    ROW_NUMBER() OVER (ORDER BY order_date) as day_number,
    RANK() OVER (ORDER BY amount DESC) as amount_rank
FROM daily_sales;
```

## Advanced Patterns

### 1. **Gap and Island Analysis**
```sql
-- Find consecutive sequences
WITH gaps AS (
    SELECT 
        id,
        id - ROW_NUMBER() OVER (ORDER BY id) as grp
    FROM sequence_table
)
SELECT 
    MIN(id) as start_id,
    MAX(id) as end_id,
    COUNT(*) as sequence_length
FROM gaps
GROUP BY grp;
```

### 2. **Median Calculation**
```sql
WITH ranked_values AS (
    SELECT 
        value,
        ROW_NUMBER() OVER (ORDER BY value) as rn,
        COUNT(*) OVER () as total_count
    FROM measurements
)
SELECT AVG(value) as median
FROM ranked_values
WHERE rn IN ((total_count + 1) / 2, (total_count + 2) / 2);
```

## When to Use Which Function

| Scenario | Use Function | Why |
|----------|--------------|-----|
| Pagination | ROW_NUMBER() | Need unique sequential numbers |
| Top N per group | ROW_NUMBER() | Guarantees exactly N rows |
| Sports rankings | RANK() | Traditional ranking with gaps |
| Grade distributions | DENSE_RANK() | Consecutive ranking levels |
| Duplicate removal | ROW_NUMBER() | Unique identifier for each row |
| Percentile analysis | RANK() or DENSE_RANK() | Depending on gap preference |

## Key Takeaways

1. **ROW_NUMBER()** is your go-to for unique sequential numbering
2. **RANK()** mimics traditional ranking systems (think Olympics)
3. **DENSE_RANK()** provides consecutive numbering without gaps
4. All three require ORDER BY clause
5. PARTITION BY creates separate ranking groups
6. Performance depends heavily on indexing strategy
7. These functions are essential for data analysis and extremely common in interviews

Remember: Master these three functions, and you'll solve 80% of ranking-related problems in SQL!

# ___________________________________________________________________________________________________________________

# LAG() and LEAD() Functions - Complete Deep Dive

## The Core Concept: Looking Backward and Forward

LAG() and LEAD() are window functions that let you **access data from other rows** without using self-joins. They're like having a time machine for your data - you can look at previous or future rows while staying in the current row.

### The Mental Model
Imagine you're looking at a spreadsheet:
- **LAG()**: "What was the value in the row above me?"
- **LEAD()**: "What's the value in the row below me?"

But unlike spreadsheet formulas, these work with proper ordering and partitioning logic.

## Function Syntax & Core Behavior

```sql
LAG(column, offset, default_value) OVER (PARTITION BY column ORDER BY column)
LEAD(column, offset, default_value) OVER (PARTITION BY column ORDER BY column)
```

### Parameters:
- **column**: The column value you want to retrieve
- **offset**: How many rows back/forward (default = 1)
- **default_value**: What to return if no row exists (default = NULL)
- **OVER()**: Defines the window scope

### Key Insight: The "Tiebreaker Principle" Applies Here Too!
Just like with ROW_NUMBER(), if your ORDER BY isn't unique, LAG/LEAD results become unpredictable. Always ensure deterministic ordering.

## Detailed Examples

### Example 1: Sales Trend Analysis
```sql
-- Sample data: daily_sales table
CREATE TABLE daily_sales (
    sale_date DATE,
    store_id INT,
    revenue DECIMAL(10,2)
);

-- Compare each day to previous day
SELECT 
    sale_date,
    store_id,
    revenue,
    LAG(revenue, 1) OVER (PARTITION BY store_id ORDER BY sale_date) as prev_day_revenue,
    LEAD(revenue, 1) OVER (PARTITION BY store_id ORDER BY sale_date) as next_day_revenue,
    revenue - LAG(revenue, 1) OVER (PARTITION BY store_id ORDER BY sale_date) as daily_change,
    ROUND(
        (revenue - LAG(revenue, 1) OVER (PARTITION BY store_id ORDER BY sale_date)) * 100.0 / 
        LAG(revenue, 1) OVER (PARTITION BY store_id ORDER BY sale_date), 2
    ) as pct_change
FROM daily_sales
ORDER BY store_id, sale_date;
```

**Sample Result:**
| sale_date | store_id | revenue | prev_day_revenue | next_day_revenue | daily_change | pct_change |
|-----------|----------|---------|------------------|------------------|--------------|------------|
| 2024-01-01| 1        | 1000    | NULL             | 1200             | NULL         | NULL       |
| 2024-01-02| 1        | 1200    | 1000             | 950              | 200          | 20.00      |
| 2024-01-03| 1        | 950     | 1200             | NULL             | -250         | -20.83     |

## Popular Use Cases

### 1. **Customer Journey Analysis**
```sql
-- Track customer behavior across sessions
WITH customer_sessions AS (
    SELECT 
        customer_id,
        session_date,
        pages_viewed,
        session_duration,
        LAG(session_date) OVER (PARTITION BY customer_id ORDER BY session_date) as prev_session_date,
        LAG(pages_viewed) OVER (PARTITION BY customer_id ORDER BY session_date) as prev_pages_viewed
    FROM user_sessions
)
SELECT 
    customer_id,
    session_date,
    pages_viewed,
    prev_pages_viewed,
    pages_viewed - prev_pages_viewed as engagement_change,
    session_date - prev_session_date as days_between_sessions
FROM customer_sessions
WHERE prev_session_date IS NOT NULL;
```

### 2. **Stock Price Analysis**
```sql
-- Calculate moving differences and identify patterns
SELECT 
    ticker,
    trade_date,
    closing_price,
    LAG(closing_price, 1) OVER (PARTITION BY ticker ORDER BY trade_date) as prev_close,
    LAG(closing_price, 7) OVER (PARTITION BY ticker ORDER BY trade_date) as week_ago_close,
    closing_price - LAG(closing_price, 1) OVER (PARTITION BY ticker ORDER BY trade_date) as daily_change,
    closing_price - LAG(closing_price, 7) OVER (PARTITION BY ticker ORDER BY trade_date) as weekly_change,
    CASE 
        When closing_price > LAG(closing_price, 1) OVER (PARTITION BY ticker ORDER BY trade_date) 
        THEN 'UP'
        WHEN closing_price < LAG(closing_price, 1) OVER (PARTITION BY ticker ORDER BY trade_date) 
        THEN 'DOWN'
        ELSE 'FLAT'
    END as daily_trend
FROM stock_prices;
```

### 3. **Gap Detection in Sequences**
```sql
-- Find missing dates in a time series
WITH date_gaps AS (
    SELECT 
        event_date,
        LAG(event_date) OVER (ORDER BY event_date) as prev_date,
        event_date - LAG(event_date) OVER (ORDER BY event_date) as gap_days
    FROM events
)
SELECT 
    prev_date,
    event_date,
    gap_days
FROM date_gaps
WHERE gap_days > 1  -- More than 1 day gap
ORDER BY event_date;
```

### 4. **Funnel Analysis**
```sql
-- Track user progression through conversion funnel
WITH user_funnel AS (
    SELECT 
        user_id,
        step_name,
        step_timestamp,
        LAG(step_name) OVER (PARTITION BY user_id ORDER BY step_timestamp) as prev_step,
        LAG(step_timestamp) OVER (PARTITION BY user_id ORDER BY step_timestamp) as prev_timestamp,
        LEAD(step_name) OVER (PARTITION BY user_id ORDER BY step_timestamp) as next_step
    FROM conversion_events
)
SELECT 
    user_id,
    prev_step,
    step_name,
    next_step,
    step_timestamp - prev_timestamp as time_between_steps,
    CASE 
        WHEN next_step IS NULL THEN 'DROP_OFF'
        ELSE 'CONTINUED'
    END as funnel_outcome
FROM user_funnel;
```

## Advanced Patterns

### 1. **Running Streaks**
```sql
-- Find consecutive days with increasing sales
WITH sales_trends AS (
    SELECT 
        sale_date,
        revenue,
        LAG(revenue) OVER (ORDER BY sale_date) as prev_revenue,
        CASE 
            WHEN revenue > LAG(revenue) OVER (ORDER BY sale_date) THEN 1 
            ELSE 0 
        END as is_increase
    FROM daily_sales
),
streak_groups AS (
    SELECT 
        sale_date,
        revenue,
        is_increase,
        SUM(CASE WHEN is_increase = 0 THEN 1 ELSE 0 END) 
            OVER (ORDER BY sale_date) as streak_group
    FROM sales_trends
)
SELECT 
    streak_group,
    MIN(sale_date) as streak_start,
    MAX(sale_date) as streak_end,
    COUNT(*) as streak_length
FROM streak_groups
WHERE is_increase = 1
GROUP BY streak_group
HAVING COUNT(*) >= 3  -- Streaks of 3+ days
ORDER BY streak_length DESC;
```

### 2. **First and Last Value in Groups**
```sql
-- Compare current value to first and last in each group
SELECT 
    department,
    employee_name,
    hire_date,
    salary,
    FIRST_VALUE(salary) OVER (PARTITION BY department ORDER BY hire_date) as first_hire_salary,
    LAST_VALUE(salary) OVER (
        PARTITION BY department 
        ORDER BY hire_date 
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) as last_hire_salary,
    salary - FIRST_VALUE(salary) OVER (PARTITION BY department ORDER BY hire_date) as vs_first_hire
FROM employees;
```

### 3. **Multi-Step Lookback**
```sql
-- Compare with multiple previous periods
SELECT 
    month_year,
    revenue,
    LAG(revenue, 1) OVER (ORDER BY month_year) as prev_month,
    LAG(revenue, 3) OVER (ORDER BY month_year) as three_months_ago,
    LAG(revenue, 12) OVER (ORDER BY month_year) as year_ago,
    -- Month-over-month growth
    (revenue - LAG(revenue, 1) OVER (ORDER BY month_year)) * 100.0 / 
        LAG(revenue, 1) OVER (ORDER BY month_year) as mom_growth,
    -- Year-over-year growth
    (revenue - LAG(revenue, 12) OVER (ORDER BY month_year)) * 100.0 / 
        LAG(revenue, 12) OVER (ORDER BY month_year) as yoy_growth
FROM monthly_revenue;
```

## Python Pandas Equivalents

### Basic LAG/LEAD
```python
# SQL: LAG(column) OVER (ORDER BY date)
df['prev_value'] = df.sort_values('date')['value'].shift(1)

# SQL: LEAD(column) OVER (ORDER BY date)  
df['next_value'] = df.sort_values('date')['value'].shift(-1)

# With grouping
df['prev_value'] = df.sort_values('date').groupby('customer_id')['value'].shift(1)
```

### Advanced Operations
```python
# Multiple period shifts
df = df.sort_values('date')
df['prev_month'] = df['revenue'].shift(1)
df['three_months_ago'] = df['revenue'].shift(3)
df['year_ago'] = df['revenue'].shift(12)

# Percentage changes
df['mom_pct'] = df['revenue'].pct_change() * 100
df['yoy_pct'] = df['revenue'].pct_change(periods=12) * 100

# With default values
df['prev_value'] = df['value'].shift(1).fillna(0)  # Default to 0
```

## Excel Equivalents

### Basic References
```excel
# LAG equivalent (previous row)
=OFFSET(B2,-1,0)  # Gets value from row above

# LEAD equivalent (next row)  
=OFFSET(B2,1,0)   # Gets value from row below

# With error handling
=IFERROR(OFFSET(B2,-1,0), 0)  # Default to 0 if no previous row
```

### Percentage Change
```excel
# Month-over-month growth
=(B2-B1)/B1*100

# With error handling for first row
=IF(ROW()=2, "", (B2-B1)/B1*100)
```

## Performance Considerations

### 1. **Index Strategy**
```sql
-- Create composite indexes for PARTITION BY + ORDER BY
CREATE INDEX idx_sales_store_date ON daily_sales(store_id, sale_date);
CREATE INDEX idx_events_user_timestamp ON user_events(user_id, event_timestamp);
```

### 2. **Window Frame Optimization**
```sql
-- Default frame is usually efficient
LAG(revenue) OVER (PARTITION BY store_id ORDER BY sale_date)

-- Avoid unnecessary frame specifications
-- This is redundant for LAG/LEAD:
LAG(revenue) OVER (
    PARTITION BY store_id 
    ORDER BY sale_date 
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW  -- Unnecessary!
)
```

### 3. **Reusing Window Definitions**
```sql
-- Instead of repeating the OVER clause:
SELECT 
    sale_date,
    revenue,
    LAG(revenue, 1) OVER (PARTITION BY store_id ORDER BY sale_date) as prev_day,
    LAG(revenue, 7) OVER (PARTITION BY store_id ORDER BY sale_date) as week_ago,
    LEAD(revenue, 1) OVER (PARTITION BY store_id ORDER BY sale_date) as next_day
FROM daily_sales;

-- Use WINDOW clause:
SELECT 
    sale_date,
    revenue,
    LAG(revenue, 1) OVER w as prev_day,
    LAG(revenue, 7) OVER w as week_ago,
    LEAD(revenue, 1) OVER w as next_day
FROM daily_sales
WINDOW w AS (PARTITION BY store_id ORDER BY sale_date);
```

## Common Interview Questions

### Q1: "Calculate month-over-month growth rate"
```sql
WITH monthly_growth AS (
    SELECT 
        month_year,
        revenue,
        LAG(revenue) OVER (ORDER BY month_year) as prev_month_revenue
    FROM monthly_sales
)
SELECT 
    month_year,
    revenue,
    prev_month_revenue,
    ROUND(
        (revenue - prev_month_revenue) * 100.0 / prev_month_revenue, 2
    ) as growth_rate_pct
FROM monthly_growth
WHERE prev_month_revenue IS NOT NULL;
```

### Q2: "Find the longest streak of increasing values"
```sql
WITH value_changes AS (
    SELECT 
        date_col,
        value,
        CASE 
            WHEN value > LAG(value) OVER (ORDER BY date_col) THEN 1 
            ELSE 0 
        END as is_increase
    FROM time_series
),
streak_groups AS (
    SELECT 
        date_col,
        value,
        is_increase,
        SUM(CASE WHEN is_increase = 0 THEN 1 ELSE 0 END) 
            OVER (ORDER BY date_col) as group_id
    FROM value_changes
)
SELECT 
    group_id,
    COUNT(*) as streak_length,
    MIN(date_col) as streak_start,
    MAX(date_col) as streak_end
FROM streak_groups
WHERE is_increase = 1
GROUP BY group_id
ORDER BY streak_length DESC
LIMIT 1;
```

### Q3: "Identify customer churn (no activity for 30+ days)"
```sql
WITH customer_activity AS (
    SELECT 
        customer_id,
        activity_date,
        LEAD(activity_date) OVER (
            PARTITION BY customer_id 
            ORDER BY activity_date
        ) as next_activity_date
    FROM customer_events
)
SELECT 
    customer_id,
    activity_date as last_activity,
    next_activity_date,
    next_activity_date - activity_date as days_inactive
FROM customer_activity
WHERE next_activity_date - activity_date > 30
   OR next_activity_date IS NULL;  -- No future activity
```

## When to Use LAG vs LEAD

| Scenario | Use Function | Example |
|----------|--------------|---------|
| Trend analysis | LAG() | Compare to previous period |
| Growth calculations | LAG() | Current vs previous value |
| Forecasting validation | LEAD() | Compare prediction to actual |
| Pipeline analysis | LEAD() | What happens next? |
| Cohort analysis | Both | Entry and exit behaviors |
| Time series gaps | LAG() | Find missing periods |

## Key Mental Models

1. **LAG() = "What happened before?"** - Perfect for trend analysis, growth calculations
2. **LEAD() = "What happens next?"** - Great for predicting outcomes, validating forecasts
3. **Both preserve row context** - Unlike self-joins, you stay in the current row while accessing others
4. **Offset parameter is powerful** - Look back/forward multiple periods easily
5. **Default values handle boundaries** - Control what happens at start/end of partitions

## The "Tiebreaker" Connection

Just like with ROW_NUMBER(), deterministic ordering is crucial:
```sql
-- ❌ Unpredictable if multiple records per day
LAG(revenue) OVER (PARTITION BY store_id ORDER BY sale_date)

-- ✅ Predictable with tiebreaker
LAG(revenue) OVER (PARTITION BY store_id ORDER BY sale_date, transaction_id)
```

## Key Takeaways

1. **LAG/LEAD eliminate complex self-joins** for accessing neighboring rows
2. **Essential for time series analysis** - trends, growth, patterns
3. **Offset parameter** lets you look multiple periods back/forward
4. **Default values** handle edge cases gracefully
5. **Performance depends on proper indexing** of partition and order columns
6. **Always ensure deterministic ordering** to avoid unpredictable results
7. **Combine with other window functions** for powerful analytics

These functions turn complex temporal analysis into simple, readable queries. Master them, and you'll solve most time-based data problems elegantly!

# ___________________________________________________________________________________________________________________

# Window Functions: first_value(), last_value(), and nth_value()

These are **window functions** in SQL that allow you to access values from other rows within a defined "window" or partition of your result set, without collapsing the data like traditional aggregate functions would.

**Core Purpose**: They retrieve specific positional values from an ordered set of rows while maintaining the granularity of your original dataset. This is crucial for comparative analysis, trend identification, and creating calculated fields that reference other rows in the same group.

**Data Analysis Applications**:
- **Baseline Comparisons**: Compare current values against first/last values in a time series
- **Performance Benchmarking**: Reference top performers or initial states
- **Gap Analysis**: Calculate differences from starting points or targets
- **Cohort Analysis**: Track changes from initial user behavior
- **Financial Analysis**: Compare against period starts, peaks, or specific benchmarks

## SQL Examples: Simple to Advanced

### Basic Usage
```sql
-- Simple: Get first and last salary in each department
SELECT 
    employee_id,
    department,
    salary,
    FIRST_VALUE(salary) OVER (PARTITION BY department ORDER BY salary DESC) as highest_salary,
    LAST_VALUE(salary) OVER (PARTITION BY department ORDER BY salary DESC 
                           ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as lowest_salary
FROM employees;
```

### Time Series Analysis
```sql
-- Compare monthly sales to first month and previous month
SELECT 
    month,
    sales,
    FIRST_VALUE(sales) OVER (ORDER BY month) as baseline_sales,
    sales - FIRST_VALUE(sales) OVER (ORDER BY month) as growth_from_start,
    NTH_VALUE(sales, 2) OVER (ORDER BY month ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as second_month_sales
FROM monthly_sales
ORDER BY month;
```

### Advanced: Customer Journey Analysis
```sql
-- Track customer behavior from first purchase
WITH customer_orders AS (
    SELECT 
        customer_id,
        order_date,
        order_value,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY order_date) as order_sequence
    FROM orders
)
SELECT 
    customer_id,
    order_date,
    order_value,
    FIRST_VALUE(order_value) OVER (PARTITION BY customer_id ORDER BY order_date) as first_order_value,
    LAST_VALUE(order_value) OVER (PARTITION BY customer_id ORDER BY order_date 
                                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as latest_order_value,
    NTH_VALUE(order_value, 3) OVER (PARTITION BY customer_id ORDER BY order_date) as third_order_value,
    order_value / FIRST_VALUE(order_value) OVER (PARTITION BY customer_id ORDER BY order_date) as value_ratio_to_first
FROM customer_orders;
```

## Python Parallels

```python
import pandas as pd

# SQL FIRST_VALUE/LAST_VALUE equivalent
df['first_salary'] = df.groupby('department')['salary'].transform('first')
df['last_salary'] = df.groupby('department')['salary'].transform('last')

# More precise control with sort
df_sorted = df.sort_values(['department', 'salary'], ascending=[True, False])
df_sorted['highest_salary'] = df_sorted.groupby('department')['salary'].transform('first')

# NTH_VALUE equivalent
df['third_value'] = df.groupby('department')['salary'].transform(lambda x: x.iloc[2] if len(x) > 2 else None)

# Time series baseline comparison
df['baseline_sales'] = df['sales'].iloc[0]  # First value
df['growth_from_start'] = df['sales'] - df['baseline_sales']

# Using shift for more complex window operations
df['previous_month'] = df['sales'].shift(1)
df['first_month'] = df.groupby('customer_id')['sales'].transform('first')
```

## Excel Equivalents

```excel
# FIRST_VALUE equivalent
=INDEX($B$2:$B$10,1)  # First value in range
=INDEX(B:B,MATCH(MIN(A:A),A:A,0))  # First value by date

# LAST_VALUE equivalent  
=INDEX($B$2:$B$10,COUNTA($B$2:$B$10))  # Last non-empty value
=LOOKUP(2,1/(B:B<>""),B:B)  # Last value alternative

# NTH_VALUE equivalent
=INDEX($B$2:$B$10,3)  # Third value

# Dynamic with OFFSET
=OFFSET($B$1,1,0)  # First value after header
=OFFSET($B$1,COUNTA($B:$B)-1,0)  # Last value

# Array formulas for grouped operations
{=INDEX(B:B,SMALL(IF(A:A=D2,ROW(A:A)),1))}  # First value in group
```

## Why These Functions Matter

**Versus Traditional Alternatives**:

1. **Better than Self-Joins**: Self-joins to get first/last values create complex, performance-heavy queries. Window functions are cleaner and faster.

2. **Superior to Subqueries**: Instead of correlated subqueries that execute repeatedly, window functions calculate once per partition.

3. **More Flexible than GROUP BY**: Unlike aggregates that collapse data, these maintain row-level detail while adding comparative context.

4. **Cleaner than Variable Assignments**: No need for complex variable logic or multiple query passes.

**Key Advantages**:
- **Performance**: Single pass through data vs multiple queries
- **Readability**: Intent is clear and declarative
- **Maintainability**: Less complex logic, fewer moving parts  
- **Analytical Power**: Enable sophisticated comparisons without data restructuring

These functions are essential for modern analytics because they bridge the gap between row-level detail and aggregate insights, enabling nuanced analysis that would otherwise require complex workarounds or multiple query steps. They're particularly powerful in time-series analysis, cohort studies, and any scenario where you need to reference "anchor points" within your data while preserving granularity.

# ___________________________________________________________________________________________________________________

# Custom Window Frames: ROWS vs RANGE

Window frames define **which rows** within your partition are included in the calculation for each current row. The key difference is **how they count**:

- **ROWS**: Counts by **physical row positions** (1st row, 2nd row, etc.)
- **RANGE**: Counts by **logical value ranges** (all rows with same value, values within X units)

## The Critical Difference

**Input Data:**
```
date       | sales | day_number
-----------|-------|------------
2024-01-01 | 100   | 1
2024-01-02 | 150   | 2  
2024-01-03 | 150   | 3  -- Same value as day 2
2024-01-04 | 200   | 4
2024-01-05 | 180   | 5
```

## ROWS Frame Examples

**ROWS**: Physical position counting
```sql
SELECT 
    date,
    sales,
    -- Last 3 physical rows (including current)
    SUM(sales) OVER (ORDER BY date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as rolling_3day_sum,
    -- Next 2 physical rows  
    AVG(sales) OVER (ORDER BY date ROWS BETWEEN CURRENT ROW AND 2 FOLLOWING) as next_2day_avg
FROM daily_sales;
```

**Output:**
```
date       | sales | rolling_3day_sum | next_2day_avg
-----------|-------|------------------|---------------
2024-01-01 | 100   | 100             | 133.33  -- (100+150+150)/3
2024-01-02 | 150   | 250             | 166.67  -- (150+150+200)/3  
2024-01-03 | 150   | 400             | 176.67  -- (150+200+180)/3
2024-01-04 | 200   | 500             | 190     -- (200+180)/2
2024-01-05 | 180   | 530             | 180     -- just current row
```

## RANGE Frame Examples

**RANGE**: Logical value counting
```sql
SELECT 
    date,
    sales,
    -- All rows with sales values within 50 of current row's value
    COUNT(*) OVER (ORDER BY sales RANGE BETWEEN 50 PRECEDING AND 50 FOLLOWING) as similar_sales_count,
    -- All rows with same sales value
    COUNT(*) OVER (ORDER BY sales RANGE BETWEEN CURRENT ROW AND CURRENT ROW) as same_value_count
FROM daily_sales;
```

**Output (ordered by sales):**
```
sales | similar_sales_count | same_value_count
------|-------------------|------------------
100   | 2                 | 1    -- Within 50: 100,150,150
150   | 4                 | 2    -- Within 50: 100,150,150,200  
150   | 4                 | 2    -- Same as above (both 150s get same result)
180   | 2                 | 1    -- Within 50: 150,150,180,200 but only 180,200 in range
200   | 2                 | 1    -- Within 50: 150,150,180,200 but only 180,200 in range
```

## Advanced Examples

### Time-Based Analysis: ROWS vs RANGE

**Input Data:**
```
date       | sales
-----------|------
2024-01-01 | 100
2024-01-03 | 200  -- Skipped day 2
2024-01-04 | 150
2024-01-07 | 300  -- Skipped days 5,6
2024-01-08 | 250
```

**ROWS Frame (Physical):**
```sql
SELECT 
    date,
    sales,
    -- Last 3 physical rows
    AVG(sales) OVER (ORDER BY date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as avg_last_3_records
FROM sales;
```

**RANGE Frame (Logical):**
```sql
SELECT 
    date,
    sales,
    -- Last 3 days of data (even with gaps)
    AVG(sales) OVER (ORDER BY date RANGE BETWEEN INTERVAL '2 days' PRECEDING AND CURRENT ROW) as avg_last_3_days
FROM sales;
```

**Comparison Output:**
```
date       | sales | avg_last_3_records | avg_last_3_days
-----------|-------|-------------------|------------------
2024-01-01 | 100   | 100               | 100      -- Only 1 row
2024-01-03 | 200   | 150               | 150      -- 2 rows within 3 days
2024-01-04 | 150   | 150               | 175      -- 3 rows: (200+150+150)/3  
2024-01-07 | 300   | 217               | 300      -- Only current (others outside 3-day window)
2024-01-08 | 250   | 233               | 275      -- 2 rows: (300+250)/2
```

## Python Equivalents

```python
import pandas as pd

# ROWS equivalent - physical positions
df['rolling_3_rows'] = df['sales'].rolling(window=3, min_periods=1).sum()

# RANGE equivalent - time-based windows  
df_indexed = df.set_index('date')
df_indexed['rolling_3_days'] = df_indexed['sales'].rolling('3D', min_periods=1).mean()

# Custom RANGE logic - value-based windows
def value_based_window(series, current_idx, range_val):
    current_val = series.iloc[current_idx]
    mask = (series >= current_val - range_val) & (series <= current_val + range_val)
    return series[mask].count()

df['similar_values'] = [value_based_window(df['sales'], i, 50) for i in range(len(df))]
```

## Excel Equivalents

```excel
# ROWS equivalent (physical positions)
=AVERAGE(OFFSET(B2,-2,0,MIN(ROW(B2)-1,3),1))  # Last 3 rows

# RANGE equivalent (date-based)  
=AVERAGEIFS(B:B,A:A,">="&A2-2,A:A,"<="&A2)    # Within 2 days

# Value-based RANGE
=COUNTIFS(B:B,">="&B2-50,B:B,"<="&B2+50)      # Values within ±50
```

## When to Use Each

### Use ROWS when:
- **Fixed number of records**: "Last 5 transactions"
- **Sequential analysis**: Moving averages over fixed periods
- **Physical positioning matters**: "Compare with previous 3 entries"

```sql
-- Moving average over last 5 sales records
SELECT 
    date,
    sales,
    AVG(sales) OVER (ORDER BY date ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) as ma_5_records
FROM sales;
```

### Use RANGE when:
- **Time-based windows**: "Last 30 days" (regardless of record count)
- **Value-based analysis**: "All products within $100 price range"  
- **Logical groupings**: "Same category or similar values"

```sql
-- Average sales within last 30 days
SELECT 
    date,
    sales,
    AVG(sales) OVER (ORDER BY date RANGE BETWEEN INTERVAL '30 days' PRECEDING AND CURRENT ROW) as avg_30_days
FROM sales;
```

## Key Advantages

**ROWS Advantages**:
- **Predictable size**: Always includes exactly N rows (when available)
- **Performance**: Simpler calculation, often faster
- **Consistent**: Same logic regardless of data distribution

**RANGE Advantages**:
- **Business logic**: Matches real-world time periods or value ranges
- **Handles gaps**: Automatically accounts for missing dates/values
- **Flexible**: Adapts to data density variations

The choice between ROWS and RANGE fundamentally depends on whether you're thinking in terms of **record count** (ROWS) or **business logic** (RANGE). Most financial and time-series analysis benefits from RANGE, while sequential record analysis often needs ROWS.

# ___________________________________________________________________________________________________________________

# Partitioning Strategies and Performance

**Partitioning** in window functions determines **how your data is divided into separate groups** before applying the window function. It's the difference between analyzing your entire dataset as one unit versus breaking it into logical segments.

**Core Purpose**: Partitioning allows you to perform calculations within meaningful business groups while maintaining row-level detail. It's crucial for comparative analysis where context matters - comparing sales reps within their regions, not globally.

**Performance Impact**: Proper partitioning can dramatically improve query performance by reducing the working set size for each calculation and enabling better use of indexes and parallel processing.

# Window Function Partitioning Examples

## Example Dataset
```
employee_id | department | region | salary | hire_date
------------|------------|--------|--------|----------
101         | Sales      | North  | 65000  | 2023-01-15
102         | Sales      | North  | 72000  | 2023-03-20
103         | Sales      | South  | 58000  | 2023-02-10
104         | Marketing  | North  | 68000  | 2023-01-25
105         | Marketing  | South  | 71000  | 2023-04-12
106         | Engineering| North  | 85000  | 2023-02-28
107         | Engineering| South  | 90000  | 2023-03-15
108         | Sales      | South  | 62000  | 2023-05-08
```

## Partitioning Strategy Examples

### 1. No Partitioning (Global Analysis)
```sql
SELECT 
    employee_id,
    department,
    salary,
    -- Global ranking across all employees
    ROW_NUMBER() OVER (ORDER BY salary DESC) as global_rank,
    -- Global salary percentile
    PERCENT_RANK() OVER (ORDER BY salary) as global_percentile
FROM employees;
```

**Output:**
```
employee_id | department  | salary | global_rank | global_percentile
------------|-------------|--------|-------------|------------------
107         | Engineering | 90000  | 1           | 1.00
106         | Engineering | 85000  | 2           | 0.86
102         | Sales       | 72000  | 3           | 0.71
105         | Marketing   | 71000  | 4           | 0.57
104         | Marketing   | 68000  | 5           | 0.43
101         | Sales       | 65000  | 6           | 0.29
108         | Sales       | 62000  | 7           | 0.14
103         | Sales       | 58000  | 8           | 0.00
```

### 2. Single Column Partitioning
```sql
SELECT 
    employee_id,
    department,
    salary,
    -- Ranking within each department
    ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary DESC) as dept_rank,
    -- Department salary percentile
    PERCENT_RANK() OVER (PARTITION BY department ORDER BY salary) as dept_percentile,
    -- Average salary in department
    AVG(salary) OVER (PARTITION BY department) as dept_avg_salary
FROM employees;
```

**Output:**
```
employee_id | department  | salary | dept_rank | dept_percentile | dept_avg_salary
------------|-------------|--------|-----------|-----------------|----------------
107         | Engineering | 90000  | 1         | 1.00            | 87500
106         | Engineering | 85000  | 2         | 0.00            | 87500
105         | Marketing   | 71000  | 1         | 1.00            | 69500
104         | Marketing   | 68000  | 2         | 0.00            | 69500
102         | Sales       | 72000  | 1         | 1.00            | 64250
101         | Sales       | 65000  | 2         | 0.67            | 64250
108         | Sales       | 62000  | 3         | 0.33            | 64250
103         | Sales       | 58000  | 4         | 0.00            | 64250
```

### 3. Multi-Column Partitioning
```sql
SELECT 
    employee_id,
    department,
    region,
    salary,
    -- Ranking within department AND region
    ROW_NUMBER() OVER (PARTITION BY department, region ORDER BY salary DESC) as dept_region_rank,
    -- Count of employees in same dept/region
    COUNT(*) OVER (PARTITION BY department, region) as group_size,
    -- Salary difference from department/region average
    salary - AVG(salary) OVER (PARTITION BY department, region) as diff_from_group_avg
FROM employees;
```

**Output:**
```
employee_id | department  | region | salary | dept_region_rank | group_size | diff_from_group_avg
------------|-------------|--------|--------|------------------|------------|--------------------
106         | Engineering | North  | 85000  | 1                | 1          | 0
107         | Engineering | South  | 90000  | 1                | 1          | 0
104         | Marketing   | North  | 68000  | 1                | 1          | 0
105         | Marketing   | South  | 71000  | 1                | 1          | 0
102         | Sales       | North  | 72000  | 1                | 1          | 0
101         | Sales       | South  | 65000  | 1                | 3          | 3333.33
108         | Sales       | South  | 62000  | 2                | 3          | 333.33
103         | Sales       | South  | 58000  | 3                | 3          | -3666.67
```

## Performance Optimization Strategies

### 1. Index-Aligned Partitioning
```sql
-- BAD: Partitioning doesn't match available indexes
SELECT 
    product_id,
    sale_date,
    amount,
    SUM(amount) OVER (PARTITION BY EXTRACT(MONTH FROM sale_date) ORDER BY sale_date) as monthly_running_total
FROM sales; -- Index on (product_id, sale_date)

-- GOOD: Partitioning aligns with index
SELECT 
    product_id,
    sale_date,
    amount,
    SUM(amount) OVER (PARTITION BY product_id ORDER BY sale_date) as product_running_total
FROM sales; -- Index on (product_id, sale_date) can be used efficiently
```

### 2. Cardinality-Aware Partitioning
```sql
-- BAD: High cardinality partitioning (many small groups)
SELECT 
    customer_id,
    order_date,
    amount,
    ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY order_date) as customer_order_sequence
FROM orders; -- 1 million customers, 5 million orders = tiny partitions

-- BETTER: Lower cardinality partitioning (fewer, larger groups)
SELECT 
    customer_id,
    order_date,
    amount,
    ROW_NUMBER() OVER (PARTITION BY customer_segment ORDER BY order_date) as segment_order_sequence
FROM orders 
JOIN customers USING (customer_id); -- 5 segments = larger, more efficient partitions
```

### 3. Memory-Efficient Partitioning
```sql
-- Consider partition size for memory usage
SELECT 
    transaction_id,
    account_id,
    transaction_date,
    amount,
    -- This could create huge partitions for active accounts
    SUM(amount) OVER (PARTITION BY account_id ORDER BY transaction_date) as running_balance,
    -- This creates smaller, more manageable partitions
    SUM(amount) OVER (PARTITION BY account_id, EXTRACT(YEAR FROM transaction_date) ORDER BY transaction_date) as yearly_running_balance
FROM transactions;
```

## Advanced Partitioning Patterns

### 1. Hierarchical Partitioning
```sql
-- Multi-level business hierarchy analysis
WITH sales_analysis AS (
    SELECT 
        rep_id,
        region,
        country,
        quarter,
        sales_amount,
        -- Global performance
        PERCENT_RANK() OVER (ORDER BY sales_amount DESC) as global_percentile,
        -- Country performance  
        PERCENT_RANK() OVER (PARTITION BY country ORDER BY sales_amount DESC) as country_percentile,
        -- Regional performance
        PERCENT_RANK() OVER (PARTITION BY region ORDER BY sales_amount DESC) as region_percentile
    FROM quarterly_sales
)
SELECT 
    *,
    CASE 
        WHEN global_percentile >= 0.9 THEN 'Global Top 10%'
        WHEN country_percentile >= 0.8 THEN 'Country Top 20%' 
        WHEN region_percentile >= 0.7 THEN 'Region Top 30%'
        ELSE 'Standard'
    END as performance_tier
FROM sales_analysis;
```

### 2. Dynamic Partitioning
```sql
-- Partition strategy changes based on data characteristics
SELECT 
    product_id,
    sale_date,
    quantity,
    -- For high-volume products: monthly partitions
    CASE 
        WHEN product_category = 'Electronics' 
        THEN SUM(quantity) OVER (PARTITION BY product_id, EXTRACT(YEAR_MONTH FROM sale_date) ORDER BY sale_date)
        -- For low-volume products: yearly partitions  
        ELSE SUM(quantity) OVER (PARTITION BY product_id, EXTRACT(YEAR FROM sale_date) ORDER BY sale_date)
    END as running_total
FROM product_sales ps
JOIN products p USING (product_id);
```

## Performance Monitoring Queries

### 1. Partition Size Analysis
```sql
-- Analyze partition sizes to optimize performance
WITH partition_stats AS (
    SELECT 
        department,
        region,
        COUNT(*) as partition_size,
        AVG(salary) as avg_salary,
        COUNT(*) OVER () as total_rows
    FROM employees
    GROUP BY department, region
)
SELECT 
    department,
    region,
    partition_size,
    partition_size * 100.0 / total_rows as pct_of_total,
    CASE 
        WHEN partition_size < 10 THEN 'Too Small - Consider Merging'
        WHEN partition_size > 10000 THEN 'Too Large - Consider Splitting'
        ELSE 'Optimal Size'
    END as size_assessment
FROM partition_stats
ORDER BY partition_size DESC;
```

## Basic Partitioning Concepts## Python Partitioning Equivalents

```python
import pandas as pd
import numpy as np

# Single column partitioning
df['dept_rank'] = df.groupby('department')['salary'].rank(method='dense', ascending=False)
df['dept_percentile'] = df.groupby('department')['salary'].rank(pct=True)

# Multi-column partitioning  
df['dept_region_rank'] = df.groupby(['department', 'region'])['salary'].rank(ascending=False)
df['group_avg'] = df.groupby(['department', 'region'])['salary'].transform('mean')

# Performance-optimized grouping
# Instead of high-cardinality grouping:
# df.groupby('customer_id')['amount'].sum()  # Many small groups

# Use lower cardinality:
df.groupby('customer_segment')['amount'].sum()  # Fewer, larger groups

# Hierarchical analysis
df['global_rank'] = df['sales'].rank(pct=True, ascending=False)
df['country_rank'] = df.groupby('country')['sales'].rank(pct=True, ascending=False)
df['region_rank'] = df.groupby('region')['sales'].rank(pct=True, ascending=False)
```

## Excel Partitioning Patterns

```excel
# Department-based partitioning using array formulas
=RANK(C2,IF($B$2:$B$100=B2,$C$2:$C$100),0)  # Rank within department

# Multi-level partitioning
=RANK(D2,IF(($B$2:$B$100=B2)*($C$2:$C$100=C2),$D$2:$D$100),0)  # Rank within dept AND region

# Performance consideration - use structured references
=RANK([@Salary],IF((Table1[Department]=[@Department])*(Table1[Region]=[@Region]),Table1[Salary]),0)
```

## Performance Best Practices

### 1. **Choose Appropriate Cardinality**
- **Low cardinality** (10-1000 groups): Better performance, more parallelization
- **High cardinality** (100k+ groups): Memory intensive, slower processing
- **Sweet spot**: 100-10,000 groups for most analyses

### 2. **Align with Indexes**
```sql
-- If you have index on (customer_id, order_date)
-- GOOD: Partition by customer_id
PARTITION BY customer_id ORDER BY order_date

-- BAD: Partition by product_id (not in index)  
PARTITION BY product_id ORDER BY order_date
```

### 3. **Consider Data Distribution**
```sql
-- Check partition sizes before implementing
SELECT 
    partition_column,
    COUNT(*) as partition_size,
    COUNT(*) * 100.0 / SUM(COUNT(*)) OVER () as pct_of_data
FROM your_table
GROUP BY partition_column
HAVING COUNT(*) > 1000  -- Flag large partitions
ORDER BY COUNT(*) DESC;
```

### 4. **Memory Management**
- **Large partitions**: May cause memory pressure
- **Many small partitions**: Overhead from context switching
- **Optimal**: Partitions with 100-10,000 rows each

## Why Partitioning Strategy Matters

**Versus No Partitioning**:
- **Business Relevance**: Comparisons within meaningful groups
- **Performance**: Smaller working sets, better parallelization
- **Accuracy**: Context-appropriate analysis (regional vs global rankings)

**Versus Poor Partitioning**:
- **Query Performance**: 10-100x faster with proper partitioning
- **Resource Usage**: Better memory and CPU utilization
- **Scalability**: Handles larger datasets effectively

**Key Advantages**:
- **Parallel Processing**: Different partitions can be processed simultaneously
- **Index Utilization**: Aligned partitioning leverages existing indexes
- **Memory Efficiency**: Smaller partition sizes reduce memory requirements
- **Business Logic**: Matches real-world analytical needs

The right partitioning strategy can transform a slow, resource-intensive query into a fast, efficient analysis. Always consider your data's cardinality, distribution, and business context when choosing partition columns. Start with business logic needs, then optimize for performance based on your specific data characteristics and infrastructure capabilities.

#

# ___________________________________________________________________________________________________________________

# Self Joins for Hierarchical Data

**Self joins** occur when a table is joined to itself, typically used to navigate **hierarchical relationships** where parent-child relationships exist within the same table. This is essential for organizational charts, category trees, geographical hierarchies, and any data structure where entities reference other entities in the same dataset.

**Core Purpose**: Self joins allow you to traverse hierarchical relationships, compare related records, and flatten tree structures into relational results. They're crucial for reporting chains, product categories, geographical rollups, and any scenario where you need to "walk up or down" a hierarchy.

**Data Analysis Applications**:
- **Organizational Analysis**: Employee-manager relationships, reporting structures
- **Product Hierarchies**: Category trees, part-subpart relationships  
- **Geographic Rollups**: City→State→Country, Territory→Region→Division
- **Time Series**: Period-over-period comparisons, trend analysis
- **Network Analysis**: Social connections, referral chains

## Basic Hierarchical Structure

**Input Data - Employee Hierarchy:**
```
employee_id | employee_name | manager_id | salary | department
------------|---------------|------------|--------|------------
1           | John CEO      | NULL       | 200000 | Executive
2           | Sarah VP      | 1          | 150000 | Sales
3           | Mike VP       | 1          | 145000 | Engineering  
4           | Lisa MGR      | 2          | 90000  | Sales
5           | Tom MGR       | 2          | 85000  | Sales
6           | Anna MGR      | 3          | 95000  | Engineering
7           | Bob REP       | 4          | 65000  | Sales
8           | Carol REP     | 4          | 70000  | Sales
9           | Dave DEV      | 6          | 80000  | Engineering
10          | Eve DEV       | 6          | 75000  | Engineering
```

## Simple to Advanced Examples

### 1. Basic Manager-Employee Relationships
```sql
-- Find each employee with their direct manager
SELECT 
    e.employee_id,
    e.employee_name as employee,
    e.salary as employee_salary,
    m.employee_name as manager,
    m.salary as manager_salary
FROM employees e
LEFT JOIN employees m ON e.manager_id = m.employee_id
ORDER BY e.employee_id;
```

**Output:**
```
employee_id | employee  | employee_salary | manager  | manager_salary
------------|-----------|-----------------|----------|---------------
1           | John CEO  | 200000         | NULL     | NULL
2           | Sarah VP  | 150000         | John CEO | 200000
3           | Mike VP   | 145000         | John CEO | 200000
4           | Lisa MGR  | 90000          | Sarah VP | 150000
5           | Tom MGR   | 85000          | Sarah VP | 150000
6           | Anna MGR  | 95000          | Mike VP  | 145000
7           | Bob REP   | 65000          | Lisa MGR | 90000
8           | Carol REP | 70000          | Lisa MGR | 90000
9           | Dave DEV  | 80000          | Anna MGR | 95000
10          | Eve DEV   | 75000          | Anna MGR | 95000
```

### 2. Multi-Level Hierarchy Navigation
```sql
-- Show employee with their manager and their manager's manager (3 levels)
SELECT 
    e.employee_name as employee,
    e.salary,
    m1.employee_name as direct_manager,
    m2.employee_name as senior_manager,
    m3.employee_name as executive
FROM employees e
LEFT JOIN employees m1 ON e.manager_id = m1.employee_id
LEFT JOIN employees m2 ON m1.manager_id = m2.employee_id  
LEFT JOIN employees m3 ON m2.manager_id = m3.employee_id
WHERE e.manager_id IS NOT NULL  -- Exclude CEO
ORDER BY e.employee_id;
```

**Output:**
```
employee     | salary | direct_manager | senior_manager | executive
-------------|--------|----------------|----------------|----------
Sarah VP     | 150000 | John CEO       | NULL           | NULL
Mike VP      | 145000 | John CEO       | NULL           | NULL
Lisa MGR     | 90000  | Sarah VP       | John CEO       | NULL
Tom MGR      | 85000  | Sarah VP       | John CEO       | NULL
Anna MGR     | 95000  | Mike VP        | John CEO       | NULL
Bob REP      | 65000  | Lisa MGR       | Sarah VP       | John CEO
Carol REP    | 70000  | Lisa MGR       | Sarah VP       | John CEO
Dave DEV     | 80000  | Anna MGR       | Mike VP        | John CEO
Eve DEV      | 75000  | Anna MGR       | Mike VP        | John CEO
```

### 3. Advanced: Hierarchical Aggregations
```sql
-- Calculate team sizes and total salary costs by manager
WITH manager_stats AS (
    SELECT 
        m.employee_id as manager_id,
        m.employee_name as manager_name,
        COUNT(e.employee_id) as direct_reports,
        SUM(e.salary) as team_salary_cost,
        AVG(e.salary) as avg_team_salary,
        MAX(e.salary) as highest_team_salary,
        MIN(e.salary) as lowest_team_salary
    FROM employees m
    INNER JOIN employees e ON m.employee_id = e.manager_id
    GROUP BY m.employee_id, m.employee_name
)
SELECT 
    manager_name,
    direct_reports,
    team_salary_cost,
    avg_team_salary,
    highest_team_salary - lowest_team_salary as salary_range,
    team_salary_cost * 100.0 / SUM(team_salary_cost) OVER () as pct_of_total_cost
FROM manager_stats
ORDER BY team_salary_cost DESC;
```

**Output:**
```
manager_name | direct_reports | team_salary_cost | avg_team_salary | salary_range | pct_of_total_cost
-------------|----------------|------------------|-----------------|--------------|------------------
John CEO     | 2              | 295000          | 147500          | 5000         | 36.8%
Sarah VP     | 2              | 175000          | 87500           | 5000         | 21.8%
Lisa MGR     | 2              | 135000          | 67500           | 5000         | 16.8%
Anna MGR     | 2              | 155000          | 77500           | 5000         | 19.3%
Mike VP      | 1              | 95000           | 95000           | 0            | 11.8%
```

### 4. Complex: Recursive Hierarchy with Path Tracking
```sql
-- Find all subordinates for each manager (direct and indirect)
WITH RECURSIVE hierarchy_path AS (
    -- Base case: Direct manager-employee relationships
    SELECT 
        manager_id,
        employee_id,
        employee_name,
        salary,
        1 as level,
        CAST(employee_name AS VARCHAR(1000)) as path
    FROM employees 
    WHERE manager_id IS NOT NULL
    
    UNION ALL
    
    -- Recursive case: Find indirect reports
    SELECT 
        hp.manager_id,
        e.employee_id,
        e.employee_name,
        e.salary,
        hp.level + 1,
        hp.path || ' -> ' || e.employee_name
    FROM hierarchy_path hp
    JOIN employees e ON hp.employee_id = e.manager_id
)
SELECT 
    m.employee_name as manager,
    COUNT(*) as total_subordinates,
    SUM(hp.salary) as total_team_cost,
    STRING_AGG(hp.employee_name || ' (L' || hp.level || ')', ', ') as all_reports
FROM employees m
JOIN hierarchy_path hp ON m.employee_id = hp.manager_id
GROUP BY m.employee_id, m.employee_name
ORDER BY total_subordinates DESC;
```

## Python Equivalents

```python
import pandas as pd
import networkx as nx

# Basic manager-employee join
df_with_managers = pd.merge(
    employees, 
    employees[['employee_id', 'employee_name', 'salary']], 
    left_on='manager_id', 
    right_on='employee_id', 
    suffixes=('', '_manager'),
    how='left'
)

# Multi-level hierarchy
def get_hierarchy_levels(df, employee_col='employee_id', manager_col='manager_id', levels=3):
    result = df.copy()
    current_df = df
    
    for level in range(1, levels + 1):
        manager_info = df[['employee_id', 'employee_name']].rename(
            columns={'employee_id': f'manager_id_l{level}', 'employee_name': f'manager_name_l{level}'}
        )
        
        current_df = pd.merge(
            current_df, 
            manager_info, 
            left_on='manager_id' if level == 1 else f'manager_id_l{level-1}',
            right_on=f'manager_id_l{level}',
            how='left'
        )
    
    return current_df

# Recursive hierarchy using NetworkX
def build_org_chart(df):
    G = nx.DiGraph()
    
    # Add edges (manager -> employee)
    for _, row in df.iterrows():
        if pd.notna(row['manager_id']):
            G.add_edge(row['manager_id'], row['employee_id'])
    
    # Find all subordinates for each manager
    subordinates = {}
    for node in G.nodes():
        subordinates[node] = list(nx.descendants(G, node))
    
    return subordinates

# Team aggregations
team_stats = employees.groupby('manager_id').agg({
    'employee_id': 'count',
    'salary': ['sum', 'mean', 'min', 'max']
}).round(2)
```

## Excel Hierarchical Patterns

```excel
# Basic manager lookup
=INDEX(employee_names, MATCH([@manager_id], employee_ids, 0))

# Multi-level hierarchy (complex but possible)
=INDEX(employee_names, MATCH(INDEX(manager_ids, MATCH([@manager_id], employee_ids, 0)), employee_ids, 0))

# Team aggregations
=SUMIF(manager_id_range, A2, salary_range)  # Total team salary
=COUNTIF(manager_id_range, A2)             # Team size

# Hierarchical path (requires helper columns)
=IF([@manager_id]="", [@employee_name], 
    INDEX(paths, MATCH([@manager_id], employee_ids, 0)) & " -> " & [@employee_name])
```

## Advanced Use Cases

### 1. Product Category Hierarchy
```sql
-- Product category rollups
WITH category_sales AS (
    SELECT 
        c1.category_id,
        c1.category_name,
        c2.category_name as parent_category,
        c3.category_name as grandparent_category,
        SUM(s.sales_amount) as category_sales
    FROM categories c1
    LEFT JOIN categories c2 ON c1.parent_category_id = c2.category_id
    LEFT JOIN categories c3 ON c2.parent_category_id = c3.category_id
    LEFT JOIN sales s ON c1.category_id = s.category_id
    GROUP BY c1.category_id, c1.category_name, c2.category_name, c3.category_name
)
SELECT 
    COALESCE(grandparent_category, parent_category, category_name) as top_level_category,
    SUM(category_sales) as total_sales,
    COUNT(*) as subcategories
FROM category_sales
GROUP BY COALESCE(grandparent_category, parent_category, category_name);
```

### 2. Geographic Territory Analysis
```sql
-- Territory hierarchy with sales rollups
SELECT 
    t1.territory_name as territory,
    t2.territory_name as region,
    t3.territory_name as division,
    COUNT(DISTINCT s.sales_rep_id) as rep_count,
    SUM(s.sales_amount) as total_sales,
    AVG(s.sales_amount) as avg_deal_size
FROM territories t1
LEFT JOIN territories t2 ON t1.parent_territory_id = t2.territory_id
LEFT JOIN territories t3 ON t2.parent_territory_id = t3.territory_id
LEFT JOIN sales s ON t1.territory_id = s.territory_id
GROUP BY t1.territory_id, t1.territory_name, t2.territory_name, t3.territory_name
HAVING SUM(s.sales_amount) > 100000
ORDER BY total_sales DESC;
```

## Why Self Joins Excel Over Alternatives

**Versus Recursive CTEs**:
- **Simplicity**: Easier to understand and debug for fixed-depth hierarchies
- **Performance**: Often faster for shallow hierarchies (2-4 levels)
- **Compatibility**: Works across more database systems

**Versus Application Logic**:
- **Efficiency**: Single query vs multiple round trips
- **Data Integrity**: Consistent view of hierarchy at query time
- **Analytical Power**: Easy to combine with aggregations and window functions

**Versus Materialized Paths**:
- **Flexibility**: No need to maintain separate path columns
- **Real-time**: Always reflects current hierarchy structure
- **Storage**: No additional storage overhead

**Key Advantages**:
- **Intuitive**: Mirrors how we think about hierarchical relationships
- **Flexible**: Can navigate up or down the hierarchy as needed
- **Powerful**: Combines easily with aggregations and analytics
- **Standard SQL**: Works across all major database platforms

Self joins for hierarchical data are fundamental for organizational reporting, product analytics, and any domain with natural parent-child relationships. They provide the foundation for understanding structure, calculating rollups, and performing comparative analysis within hierarchical contexts. Master this pattern, and you'll handle everything from org charts to product categories with confidence.

# ___________________________________________________________________________________________________________________

# Multiple Table Join Optimizations

**Join optimization** is the art and science of efficiently combining data from multiple tables while minimizing resource consumption and query execution time. With multiple tables, the complexity grows exponentially - both in terms of possible execution plans and potential performance pitfalls.

**Core Purpose**: Optimize how the database engine accesses, filters, and combines data across tables. Poor join strategies can turn millisecond queries into hour-long operations, while proper optimization makes complex multi-table analytics lightning-fast.

**Performance Impact Areas**:
- **Execution Plan Selection**: Guiding the optimizer toward efficient join orders
- **Index Utilization**: Ensuring joins leverage existing indexes effectively  
- **Memory Management**: Controlling working set sizes and temporary storage
- **I/O Reduction**: Minimizing disk reads through smart filtering and ordering

## Join Order Fundamentals
# Multiple Table Join Optimization Examples

## Sample Schema
```sql
-- Large tables
orders (5M rows)      -- order_id, customer_id, order_date, status, total_amount
customers (500K rows) -- customer_id, customer_name, segment, region, signup_date
products (50K rows)   -- product_id, product_name, category_id, price, supplier_id

-- Medium tables  
order_items (15M rows) -- order_id, product_id, quantity, unit_price
categories (500 rows)  -- category_id, category_name, parent_category_id
suppliers (1K rows)   -- supplier_id, supplier_name, country, rating

-- Small tables
regions (50 rows)     -- region_id, region_name, country_code
promotions (100 rows) -- promo_id, promo_code, discount_pct, start_date, end_date
```

## 1. Join Order Optimization

### BAD: Large Table First, No Filtering
```sql
-- Inefficient: Starts with largest table, no early filtering
SELECT 
    c.customer_name,
    p.product_name,
    oi.quantity,
    o.order_date
FROM order_items oi                    -- 15M rows - LARGEST FIRST (BAD)
JOIN orders o ON oi.order_id = o.order_id
JOIN customers c ON o.customer_id = c.customer_id
JOIN products p ON oi.product_id = p.product_id
WHERE o.order_date >= '2024-01-01'     -- Filter applied AFTER joins (BAD)
  AND c.segment = 'Premium';           -- Another late filter (BAD)
```

### GOOD: Filtered Small Tables First
```sql
-- Efficient: Start with filtered small tables, build up
SELECT 
    c.customer_name,
    p.product_name,
    oi.quantity,
    o.order_date
FROM customers c                        -- Start with customers (500K)
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
WHERE c.segment = 'Premium'             -- Filter early (reduces customer set)
  AND o.order_date >= '2024-01-01'      -- Filter early (reduces order set)
  AND o.status = 'completed';           -- Additional early filter
```

### OPTIMAL: Use CTEs for Complex Filtering
```sql
-- Best: Pre-filter tables, then join smaller result sets
WITH premium_customers AS (
    SELECT customer_id, customer_name 
    FROM customers 
    WHERE segment = 'Premium'           -- 50K customers instead of 500K
),
recent_orders AS (
    SELECT order_id, customer_id, order_date
    FROM orders 
    WHERE order_date >= '2024-01-01'    -- 1M orders instead of 5M
      AND status = 'completed'
),
relevant_items AS (
    SELECT oi.order_id, oi.product_id, oi.quantity
    FROM order_items oi
    WHERE EXISTS (
        SELECT 1 FROM recent_orders ro 
        WHERE ro.order_id = oi.order_id -- Only items from recent orders
    )
)
SELECT 
    pc.customer_name,
    p.product_name,
    ri.quantity,
    ro.order_date
FROM premium_customers pc
JOIN recent_orders ro ON pc.customer_id = ro.customer_id
JOIN relevant_items ri ON ro.order_id = ri.order_id
JOIN products p ON ri.product_id = p.product_id;
```

## 2. Index-Driven Join Strategies

### Covering Index Optimization
```sql
-- Create covering indexes for common join patterns
CREATE INDEX idx_orders_customer_date_covering 
ON orders (customer_id, order_date) 
INCLUDE (order_id, status, total_amount);

CREATE INDEX idx_order_items_order_product_covering
ON order_items (order_id, product_id)
INCLUDE (quantity, unit_price);

-- Query that leverages covering indexes
SELECT 
    o.order_id,
    o.order_date,
    o.total_amount,
    SUM(oi.quantity * oi.unit_price) as calculated_total
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.customer_id = 12345
  AND o.order_date >= '2024-01-01'
GROUP BY o.order_id, o.order_date, o.total_amount;
```

### Composite Key Optimization
```sql
-- Optimize multi-column join conditions
CREATE INDEX idx_composite_join 
ON order_items (order_id, product_id, quantity);

-- Efficient query using composite index
SELECT 
    o.order_date,
    p.product_name,
    oi.quantity
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
WHERE o.customer_id IN (SELECT customer_id FROM premium_customers)
  AND oi.quantity > 1                  -- Can use composite index
ORDER BY o.order_date, p.product_name;
```

## 3. Join Algorithm Selection

### Hash Join Optimization (Large × Large)
```sql
-- Force hash join for large table combinations
SELECT /*+ USE_HASH(o,oi) */
    o.order_date,
    COUNT(*) as order_count,
    SUM(oi.quantity * oi.unit_price) as total_value
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.order_date BETWEEN '2024-01-01' AND '2024-03-31'
GROUP BY o.order_date
ORDER BY o.order_date;
```

### Nested Loop Optimization (Small × Large)
```sql
-- Nested loop is efficient when outer table is small
SELECT 
    r.region_name,
    c.customer_name,
    COUNT(o.order_id) as order_count
FROM regions r                          -- Small table (50 rows)
JOIN customers c ON r.region_name = c.region
JOIN orders o ON c.customer_id = o.customer_id
WHERE r.region_name IN ('California', 'Texas', 'New York')  -- Very selective
GROUP BY r.region_name, c.customer_name;
```

### Sort-Merge Join Optimization
```sql
-- When both tables are already sorted on join keys
SELECT 
    o.order_date,
    c.signup_date,
    COUNT(*) as orders
FROM orders o                           -- Assume clustered on order_date
JOIN customers c ON o.customer_id = c.customer_id  -- Assume sorted on customer_id
WHERE o.order_date >= '2024-01-01'
GROUP BY o.order_date, c.signup_date
ORDER BY o.order_date;                  -- Already in order
```

## 4. Anti-Pattern Detection and Fixes

### Problem: Cartesian Product
```sql
-- BAD: Missing join condition creates cartesian product
SELECT 
    c.customer_name,
    p.product_name,
    o.order_date
FROM customers c, products p, orders o   -- Old-style joins (dangerous)
WHERE c.segment = 'Premium'
  AND o.order_date >= '2024-01-01';
-- Results: 500K × 50K × 5M = 1.25 quadrillion rows!

-- GOOD: Proper join conditions
SELECT 
    c.customer_name,
    p.product_name,
    o.order_date
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
WHERE c.segment = 'Premium'
  AND o.order_date >= '2024-01-01';
```

### Problem: Excessive Self-Joins
```sql
-- BAD: Multiple self-joins for different time periods
SELECT 
    c.customer_name,
    o1.total_amount as q1_total,
    o2.total_amount as q2_total,
    o3.total_amount as q3_total,
    o4.total_amount as q4_total
FROM customers c
LEFT JOIN orders o1 ON c.customer_id = o1.customer_id 
    AND o1.order_date BETWEEN '2024-01-01' AND '2024-03-31'
LEFT JOIN orders o2 ON c.customer_id = o2.customer_id 
    AND o2.order_date BETWEEN '2024-04-01' AND '2024-06-30'
LEFT JOIN orders o3 ON c.customer_id = o3.customer_id 
    AND o3.order_date BETWEEN '2024-07-01' AND '2024-09-30'
LEFT JOIN orders o4 ON c.customer_id = o4.customer_id 
    AND o4.order_date BETWEEN '2024-10-01' AND '2024-12-31';

-- GOOD: Single join with conditional aggregation
SELECT 
    c.customer_name,
    SUM(CASE WHEN o.order_date BETWEEN '2024-01-01' AND '2024-03-31' 
             THEN o.total_amount END) as q1_total,
    SUM(CASE WHEN o.order_date BETWEEN '2024-04-01' AND '2024-06-30' 
             THEN o.total_amount END) as q2_total,
    SUM(CASE WHEN o.order_date BETWEEN '2024-07-01' AND '2024-09-30' 
             THEN o.total_amount END) as q3_total,
    SUM(CASE WHEN o.order_date BETWEEN '2024-10-01' AND '2024-12-31' 
             THEN o.total_amount END) as q4_total
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id 
    AND o.order_date BETWEEN '2024-01-01' AND '2024-12-31'
GROUP BY c.customer_id, c.customer_name;
```

## 5. Advanced Optimization Techniques

### Materialized Join Results
```sql
-- Create materialized view for frequently joined tables
CREATE MATERIALIZED VIEW customer_order_summary AS
SELECT 
    c.customer_id,
    c.customer_name,
    c.segment,
    c.region,
    COUNT(o.order_id) as total_orders,
    SUM(o.total_amount) as total_spent,
    MAX(o.order_date) as last_order_date,
    AVG(o.total_amount) as avg_order_value
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.customer_name, c.segment, c.region;

-- Refresh periodically
REFRESH MATERIALIZED VIEW customer_order_summary;

-- Use in complex queries
SELECT 
    cos.segment,
    cos.region,
    AVG(cos.avg_order_value) as segment_avg_order_value,
    COUNT(*) as customers_in_segment
FROM customer_order_summary cos
WHERE cos.total_orders > 5
GROUP BY cos.segment, cos.region;
```

### Lateral Joins for Correlated Subqueries
```sql
-- Replace inefficient correlated subqueries with lateral joins
-- BAD: Correlated subquery executes for each customer
SELECT 
    c.customer_name,
    (SELECT COUNT(*) FROM orders o WHERE o.customer_id = c.customer_id) as order_count,
    (SELECT MAX(o.order_date) FROM orders o WHERE o.customer_id = c.customer_id) as last_order
FROM customers c
WHERE c.segment = 'Premium';

-- GOOD: Lateral join (PostgreSQL/SQL Server)
SELECT 
    c.customer_name,
    os.order_count,
    os.last_order
FROM customers c
CROSS APPLY (
    SELECT 
        COUNT(*) as order_count,
        MAX(order_date) as last_order
    FROM orders o 
    WHERE o.customer_id = c.customer_id
) os
WHERE c.segment = 'Premium';
```

## 6. Performance Monitoring Queries

### Join Cost Analysis
```sql
-- Analyze join selectivity and cardinality
WITH join_stats AS (
    SELECT 
        'customers-orders' as join_name,
        COUNT(DISTINCT c.customer_id) as left_distinct,
        COUNT(DISTINCT o.customer_id) as right_distinct,
        COUNT(*) as join_result_rows,
        COUNT(*) * 1.0 / COUNT(DISTINCT c.customer_id) as avg_fanout
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    
    UNION ALL
    
    SELECT 
        'orders-order_items' as join_name,
        COUNT(DISTINCT o.order_id) as left_distinct,
        COUNT(DISTINCT oi.order_id) as right_distinct,
        COUNT(*) as join_result_rows,
        COUNT(*) * 1.0 / COUNT(DISTINCT o.order_id) as avg_fanout
    FROM orders o
    JOIN order_items oi ON o.order_id = oi.order_id
)
SELECT 
    join_name,
    left_distinct,
    right_distinct,
    join_result_rows,
    ROUND(avg_fanout, 2) as avg_fanout,
    CASE 
        WHEN avg_fanout > 10 THEN 'High Fanout - Consider Aggregation'
        WHEN avg_fanout < 1.1 THEN 'Low Fanout - Nearly 1:1'
        ELSE 'Moderate Fanout'
    END as fanout_assessment
FROM join_stats;
```

### Index Usage Analysis
```sql
-- Check if joins are using appropriate indexes
EXPLAIN (ANALYZE, BUFFERS) 
SELECT 
    c.customer_name,
    COUNT(o.order_id) as orders,
    SUM(o.total_amount) as total_spent
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
WHERE c.segment = 'Premium'
  AND o.order_date >= '2024-01-01'
GROUP BY c.customer_id, c.customer_name
HAVING COUNT(o.order_id) > 10;
```

## Python Join Optimization Equivalents


```python
import pandas as pd
import numpy as np

# Join order optimization - filter first, then join
# BAD: Join large dataframes first, filter later
# result = orders.merge(order_items, on='order_id') \
#               .merge(customers, on='customer_id') \
#               .query("segment == 'Premium' and order_date >= '2024-01-01'")

# GOOD: Filter first, then join smaller datasets
premium_customers = customers[customers['segment'] == 'Premium']
recent_orders = orders[orders['order_date'] >= '2024-01-01']

result = premium_customers.merge(recent_orders, on='customer_id') \
                         .merge(order_items, on='order_id') \
                         .merge(products, on='product_id')

# Index optimization - set appropriate indexes
customers.set_index('customer_id', inplace=True)
orders.set_index(['customer_id', 'order_date'], inplace=True)

# Memory-efficient joins for large datasets
def chunked_join(left_df, right_df, on, chunk_size=10000):
    """Join large dataframes in chunks to manage memory"""
    results = []
    for i in range(0, len(left_df), chunk_size):
        chunk = left_df.iloc[i:i+chunk_size]
        merged_chunk = chunk.merge(right_df, on=on, how='left')
        results.append(merged_chunk)
    return pd.concat(results, ignore_index=True)

# Use categorical data types for better join performance
customers['segment'] = customers['segment'].astype('category')
orders['status'] = orders['status'].astype('category')

# Optimize join order based on data size
def optimize_join_order(*dataframes, join_keys):
    """Sort dataframes by size for optimal join order"""
    df_sizes = [(df, len(df)) for df in dataframes]
    df_sizes.sort(key=lambda x: x[1])  # Sort by size
    
    result = df_sizes[0][0]  # Start with smallest
    for df, _ in df_sizes[1:]:
        result = result.merge(df, on=join_keys, how='inner')
    return result
```

## Excel Join Optimization Patterns

```excel
# Use structured tables for better performance
# Create Table1, Table2, etc. instead of ranges

# VLOOKUP optimization - exact match only
=VLOOKUP(A2,Table2,2,FALSE)

# INDEX-MATCH for better performance than VLOOKUP
=INDEX(Table2[Return_Column],MATCH(A2,Table2[Lookup_Column],0))

# Power Query for complex multi-table joins
# Data > Get Data > Combine Queries > Merge

# Optimize with helper columns for common lookups
# Instead of complex nested lookups, create intermediate columns

# Use pivot tables for aggregated joins
# Insert > PivotTable > Add multiple tables to data model
```

## Advanced Optimization Strategies

### 1. **Partitioned Joins**
```sql
-- Partition large tables by date for better join performance
CREATE TABLE orders_2024_q1 PARTITION OF orders 
FOR VALUES FROM ('2024-01-01') TO ('2024-04-01');

-- Queries automatically use appropriate partitions
SELECT o.*, c.customer_name
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
WHERE o.order_date BETWEEN '2024-01-01' AND '2024-03-31';
```

### 2. **Bloom Filters**
```sql
-- Use EXISTS instead of IN for better performance
-- BAD: IN with subquery
SELECT * FROM customers c
WHERE c.customer_id IN (
    SELECT DISTINCT customer_id FROM orders 
    WHERE order_date >= '2024-01-01'
);

-- GOOD: EXISTS (often uses bloom filters internally)
SELECT * FROM customers c
WHERE EXISTS (
    SELECT 1 FROM orders o 
    WHERE o.customer_id = c.customer_id 
      AND o.order_date >= '2024-01-01'
);
```

### 3. **Star Schema Optimization**
```sql
-- Optimize fact table joins by joining dimensions first
WITH dim_customers AS (
    SELECT customer_id FROM customers 
    WHERE segment = 'Premium' AND region = 'North'
),
dim_products AS (
    SELECT product_id FROM products 
    WHERE category = 'Electronics' AND price > 100
)
SELECT 
    COUNT(*) as sales_count,
    SUM(oi.quantity * oi.unit_price) as total_revenue
FROM order_items oi
JOIN dim_customers dc ON oi.customer_id = dc.customer_id
JOIN dim_products dp ON oi.product_id = dp.product_id
JOIN orders o ON oi.order_id = o.order_id
WHERE o.order_date >= '2024-01-01';
```

## Performance Monitoring and Debugging

### 1. **Execution Plan Analysis**
```sql
-- Check for table scans, nested loops, hash joins
EXPLAIN (ANALYZE, BUFFERS, FORMAT JSON)
SELECT /* your complex query */;

-- Look for:
-- - Table scans instead of index scans
-- - Hash joins on large tables
-- - High buffer usage
-- - Long execution times
```

### 2. **Join Cardinality Estimation**
```sql
-- Verify optimizer estimates vs actual
SELECT 
    'Estimate' as type,
    estimated_rows,
    estimated_cost
FROM query_plan
UNION ALL
SELECT 
    'Actual' as type,
    actual_rows,
    actual_cost
FROM query_execution;
```

## Why These Optimizations Matter

**Performance Impact**:
- **Query Speed**: 10-1000x faster execution with proper join order
- **Memory Usage**: Reduced working set sizes prevent out-of-memory errors
- **Scalability**: Techniques that work on small data continue working as data grows

**Versus Unoptimized Joins**:
- **Resource Efficiency**: Lower CPU, memory, and I/O usage
- **Concurrency**: Better performance under load with multiple users
- **Cost**: Reduced cloud compute costs from efficient resource usage

**Key Principles**:
- **Filter Early**: Reduce data volume before expensive operations
- **Join Small to Large**: Start with most selective tables
- **Leverage Indexes**: Ensure join conditions can use existing indexes
- **Monitor Performance**: Regular analysis of execution plans and costs

Multiple table join optimization is about understanding your data characteristics, query patterns, and database engine capabilities. Start with proper indexing, apply filtering early, and choose join orders that minimize intermediate result sizes. These techniques scale from simple reports to complex analytical workloads.

# ___________________________________________________________________________________________________________________

# CROSS APPLY v/s OUTER_APPLY

 **SQL Server** (also supported in **Azure Synapse**, and partially in **Oracle**).


## 🧠 Basic Concept

Both `CROSS APPLY` and `OUTER APPLY` let you join a **table-valued function (TVF)** or a **subquery** to each row of another table — kind of like a correlated join.

But:

| Feature               | `CROSS APPLY`                         | `OUTER APPLY`                       |
| --------------------- | ------------------------------------- | ----------------------------------- |
| Keeps unmatched rows? | ❌ No (acts like INNER JOIN)           | ✅ Yes (acts like LEFT JOIN)         |
| Use case              | When subquery **must return a match** | When subquery **might return NULL** |
| Performance           | Slightly faster (less work)           | Slightly slower (more rows)         |



## 🔧 Syntax

```sql
-- CROSS APPLY
SELECT a.*, b.*
FROM tableA a
CROSS APPLY (
    SELECT TOP 1 * FROM tableB b WHERE b.id = a.id ORDER BY b.date DESC
) b

-- OUTER APPLY
SELECT a.*, b.*
FROM tableA a
OUTER APPLY (
    SELECT TOP 1 * FROM tableB b WHERE b.id = a.id ORDER BY b.date DESC
) b
```



## 📊 Example

### Given:

**Customers**

| customer\_id | name    |
| ------------ | ------- |
| 1            | Alice   |
| 2            | Bob     |
| 3            | Charlie |

**Orders**

| order\_id | customer\_id | amount |
| --------- | ------------ | ------ |
| 101       | 1            | 200    |
| 102       | 1            | 150    |
| 103       | 2            | 300    |



### 🟢 `CROSS APPLY`: Only customers with orders

```sql
SELECT c.name, o.amount
FROM Customers c
CROSS APPLY (
    SELECT TOP 1 * FROM Orders o WHERE o.customer_id = c.customer_id ORDER BY o.amount DESC
) o;
```

**Result:**

| name  | amount |
| ----- | ------ |
| Alice | 200    |
| Bob   | 300    |

**Charlie is excluded** because he has no orders.



### 🟡 `OUTER APPLY`: Includes customers with no orders

```sql
SELECT c.name, o.amount
FROM Customers c
OUTER APPLY (
    SELECT TOP 1 * FROM Orders o WHERE o.customer_id = c.customer_id ORDER BY o.amount DESC
) o;
```

**Result:**

| name    | amount |
| ------- | ------ |
| Alice   | 200    |
| Bob     | 300    |
| Charlie | NULL   |

Charlie is **included** with `NULL` for `amount`.



## 💡 Use Cases

| Use Case                                       | APPLY Type                                          |
| ---------------------------------------------- | --------------------------------------------------- |
| Top N items per group                          | CROSS APPLY                                         |
| Getting latest row from a detail table         | APPLY both (depending on if you want unmatched too) |
| Conditional logic per row (dynamic subqueries) | APPLY both                                          |
| Optional related data (e.g., profile image)    | OUTER APPLY                                         |



## 🚀 When to Use

* Use **`CROSS APPLY`** when you're **sure** there's a match (better performance).
* Use **`OUTER APPLY`** when you want to **include rows even if there's no match**.



---

## INTERSECT and EXCEPT Operations

### Brief Explanation

**INTERSECT** returns only the rows that exist in both result sets (common rows), while **EXCEPT** returns rows that exist in the first result set but not in the second (difference/subtraction). Both operations automatically remove duplicates from the final result.

**Purposes:**
- Find common records between datasets (INTERSECT)
- Identify missing or unique records (EXCEPT)
- Perform set-based comparisons between tables

**Data Analysis Context:**
- Essential for data quality auditing and validation
- Critical for identifying gaps in data migration or synchronization
- Fundamental for comparative analysis between time periods, regions, or segments
- Used in data reconciliation and exception reporting

### Examples (Simple to Advanced)

**Simple Example:**
```sql
-- INTERSECT: Find products sold in both quarters
SELECT product_id FROM q1_sales
INTERSECT
SELECT product_id FROM q2_sales;

-- EXCEPT: Find products sold in Q1 but not in Q2
SELECT product_id FROM q1_sales
EXCEPT
SELECT product_id FROM q2_sales;
```

**Intermediate Example:**
```sql
-- Find customers who made purchases in both online and store channels
SELECT customer_id, customer_email
FROM online_customers
INTERSECT
SELECT customer_id, customer_email
FROM store_customers;

-- Find customers who shopped online but never visited stores
SELECT customer_id, customer_name, email
FROM online_customers
EXCEPT
SELECT customer_id, customer_name, email
FROM store_customers;
```

**Advanced Example:**
```sql
-- Complex analysis: Find high-value customers common to both regions
SELECT customer_id, SUM(order_value) as total_spent
FROM north_region_sales
WHERE order_date >= '2024-01-01'
GROUP BY customer_id
HAVING SUM(order_value) > 5000

INTERSECT

SELECT customer_id, SUM(order_value) as total_spent
FROM south_region_sales
WHERE order_date >= '2024-01-01'
GROUP BY customer_id
HAVING SUM(order_value) > 5000;

-- Find products that performed well in Region A but poorly in Region B
SELECT product_id, product_category
FROM region_a_sales
WHERE sales_rank <= 10
EXCEPT
SELECT product_id, product_category
FROM region_b_sales
WHERE sales_rank <= 50;
```

### Python and Excel Comparisons

**Python (Pandas):**
```python
# INTERSECT equivalent
df_intersect = pd.merge(df1, df2, how='inner').drop_duplicates()
# Or using set operations on indices
common_rows = df1[df1.index.isin(df2.index)]

# EXCEPT equivalent
df_except = df1[~df1.isin(df2)].dropna()
# Or more explicitly
df_except = pd.concat([df1, df2]).drop_duplicates(keep=False)

# Using set operations on specific columns
set1 = set(df1['column'].tolist())
set2 = set(df2['column'].tolist())
intersect_result = set1.intersection(set2)
except_result = set1.difference(set2)
```

**Excel:**
- **INTERSECT**: Use VLOOKUP or INDEX-MATCH to find common values, then remove duplicates
- **EXCEPT**: Use VLOOKUP with ISERROR to find values in first dataset not in second
- Power Query: Data → Merge Queries → Inner Join (INTERSECT) or Anti Join (EXCEPT)
- Advanced: Use COUNTIF formulas combined with filtering

```excel
# Excel formula approach for EXCEPT
=IF(COUNTIF(Sheet2!A:A,Sheet1!A2)=0,Sheet1!A2,"")
```

### Conclusion and Importance

**Why INTERSECT/EXCEPT are crucial:**

1. **Set Theory Power**: These operations bring mathematical set theory directly into SQL, enabling precise data comparisons that would require complex JOINs or subqueries otherwise.

2. **Data Quality Assurance**: 
   - **INTERSECT**: Validates data consistency across systems
   - **EXCEPT**: Identifies data gaps, missing records, or synchronization issues

3. **Performance Advantages Over Alternatives**:
   - **vs Complex JOINs**: More readable and often faster than LEFT JOIN with NULL checks
   - **vs EXISTS/NOT EXISTS**: Cleaner syntax for set-based operations
   - **vs IN/NOT IN**: Handles multiple columns naturally and avoids NULL-related pitfalls

4. **Database Engine Optimization**: Most modern databases optimize these operations at the engine level, making them extremely efficient for large datasets.

5. **Business Intelligence Applications**:
   - Customer retention analysis (who stayed vs. who left)
   - Product performance comparisons
   - Data migration validation
   - Regulatory compliance reporting

**Important Note**: Not all SQL databases support INTERSECT/EXCEPT (notably older MySQL versions). However, they can be simulated using JOINs, making understanding these concepts valuable even in limited environments.

**Best Practice**: Use INTERSECT/EXCEPT when you need clean, set-based comparisons. They're particularly powerful for data auditing, quality checks, and comparative analysis where you need to understand exactly what's common or different between datasets.


---

## CTEs (Common Table Expressions) and Data Transformation

### Brief Explanation

**CTEs (Common Table Expressions)** are temporary named result sets that exist only within the execution scope of a single SQL statement. They're defined using the `WITH` clause and act like temporary views or inline subqueries with names.

**Purposes:**
- Break complex queries into readable, manageable parts
- Create reusable query components within a single statement
- Enable recursive operations and hierarchical data processing
- Improve code maintainability and debugging

**Data Analysis Context:**
- Essential for multi-step data transformations and calculations
- Critical for creating clean, readable analytical queries
- Fundamental for handling complex business logic in data pipelines
- Used extensively in data warehousing and ETL processes
- Enables sophisticated window functions and ranking operations

### Examples (Simple to Advanced)

**Simple Example:**
```sql
-- Basic CTE for data filtering
WITH high_value_customers AS (
    SELECT customer_id, customer_name, total_spent
    FROM customers
    WHERE total_spent > 10000
)
SELECT customer_name, total_spent
FROM high_value_customers
ORDER BY total_spent DESC;
```

**Intermediate Example:**
```sql
-- Multiple CTEs for step-by-step transformation
WITH monthly_sales AS (
    SELECT 
        EXTRACT(MONTH FROM order_date) as month,
        product_id,
        SUM(quantity * price) as monthly_revenue
    FROM orders
    WHERE EXTRACT(YEAR FROM order_date) = 2024
    GROUP BY EXTRACT(MONTH FROM order_date), product_id
),
ranked_products AS (
    SELECT 
        month,
        product_id,
        monthly_revenue,
        ROW_NUMBER() OVER (PARTITION BY month ORDER BY monthly_revenue DESC) as rank
    FROM monthly_sales
)
SELECT 
    month,
    product_id,
    monthly_revenue,
    rank
FROM ranked_products
WHERE rank <= 5;
```

**Advanced Example:**
```sql
-- Complex data transformation with multiple CTEs
WITH customer_metrics AS (
    SELECT 
        customer_id,
        COUNT(DISTINCT order_id) as order_count,
        SUM(order_value) as total_spent,
        AVG(order_value) as avg_order_value,
        MIN(order_date) as first_order,
        MAX(order_date) as last_order
    FROM orders
    WHERE order_date >= '2024-01-01'
    GROUP BY customer_id
),
customer_segments AS (
    SELECT 
        customer_id,
        order_count,
        total_spent,
        avg_order_value,
        DATEDIFF(last_order, first_order) as customer_lifetime_days,
        CASE 
            WHEN total_spent > 5000 AND order_count > 10 THEN 'VIP'
            WHEN total_spent > 2000 AND order_count > 5 THEN 'Premium'
            WHEN total_spent > 500 THEN 'Regular'
            ELSE 'New'
        END as segment
    FROM customer_metrics
),
segment_analysis AS (
    SELECT 
        segment,
        COUNT(*) as customer_count,
        AVG(total_spent) as avg_total_spent,
        AVG(avg_order_value) as avg_order_size,
        AVG(customer_lifetime_days) as avg_lifetime_days
    FROM customer_segments
    GROUP BY segment
)
SELECT 
    segment,
    customer_count,
    ROUND(avg_total_spent, 2) as avg_total_spent,
    ROUND(avg_order_size, 2) as avg_order_size,
    ROUND(avg_lifetime_days, 0) as avg_lifetime_days,
    ROUND(100.0 * customer_count / SUM(customer_count) OVER(), 2) as segment_percentage
FROM segment_analysis
ORDER BY avg_total_spent DESC;

-- Recursive CTE example for hierarchical data
WITH RECURSIVE employee_hierarchy AS (
    -- Base case: top-level managers
    SELECT employee_id, employee_name, manager_id, 1 as level
    FROM employees
    WHERE manager_id IS NULL
    
    UNION ALL
    
    -- Recursive case: employees with managers
    SELECT e.employee_id, e.employee_name, e.manager_id, eh.level + 1
    FROM employees e
    JOIN employee_hierarchy eh ON e.manager_id = eh.employee_id
)
SELECT employee_id, employee_name, level,
       REPEAT('  ', level - 1) || employee_name as indented_name
FROM employee_hierarchy
ORDER BY level, employee_name;
```

### Python and Excel Comparisons

**Python (Pandas):**
```python
# CTE-like operations using method chaining and intermediate variables
# Step 1: Create intermediate dataset (like CTE)
high_value_customers = df[df['total_spent'] > 10000]

# Step 2: Further transformation
result = (high_value_customers
    .groupby('customer_segment')
    .agg({'total_spent': 'mean', 'customer_id': 'count'})
    .reset_index())

# Multiple transformations (equivalent to multiple CTEs)
monthly_sales = (df
    .groupby(['month', 'product_id'])
    .agg({'revenue': 'sum'})
    .reset_index())

ranked_products = (monthly_sales
    .assign(rank=lambda x: x.groupby('month')['revenue'].rank(method='dense', ascending=False))
    .query('rank <= 5'))

# Using pipe for CTE-like readability
result = (df
    .pipe(lambda x: x[x['total_spent'] > 10000])
    .pipe(lambda x: x.groupby('segment').agg({'total_spent': 'mean'}))
    .pipe(lambda x: x.reset_index()))
```

**Excel:**
- **Basic CTE equivalent**: Create intermediate calculations in helper columns
- **Power Query**: Use multiple transformation steps (similar to CTEs)
- **Named ranges**: Define reusable data ranges
- **Pivot Tables**: Multi-step aggregations with intermediate groupings

```excel
# Excel Power Query M language (similar to CTEs)
let
    Source = Excel.CurrentWorkbook(){[Name="SalesData"]}[Content],
    HighValueCustomers = Table.SelectRows(Source, each [Total_Spent] > 10000),
    GroupedData = Table.Group(HighValueCustomers, {"Segment"}, {{"AvgSpent", each List.Average([Total_Spent]), type number}})
in
    GroupedData
```

### Conclusion and Importance

**Why CTEs are crucial for data transformation:**

1. **Readability and Maintainability**: CTEs break complex queries into logical, named steps, making code self-documenting and easier to debug than nested subqueries.

2. **Performance Benefits**: 
   - Database engines can optimize CTE execution plans
   - Avoid repeated subquery execution
   - Enable better caching strategies

3. **Advantages Over Alternatives**:
   - **vs Subqueries**: More readable, reusable within the same query
   - **vs Views**: Temporary scope, no database object creation needed
   - **vs Temporary Tables**: No storage overhead, automatic cleanup

4. **Advanced Capabilities**:
   - **Recursive CTEs**: Handle hierarchical data (org charts, category trees) impossible with standard SQL
   - **Window Functions**: Enable sophisticated analytical calculations with clean syntax

5. **Data Pipeline Integration**: CTEs are essential in modern data warehousing:
   - **dbt (data build tool)**: Heavily relies on CTEs for transformation logic
   - **Analytical databases**: Optimized for CTE-heavy workloads
   - **Data modeling**: Enable complex business logic implementation

6. **Business Intelligence Power**: 
   - Multi-step customer segmentation
   - Complex financial calculations
   - Time-series analysis with multiple aggregation levels
   - Data quality checks and transformations

**Best Practices**: 
- Use CTEs for complex multi-step transformations
- Name CTEs descriptively to document business logic
- Combine with window functions for advanced analytics
- Prefer CTEs over nested subqueries for readability
- Use recursive CTEs for hierarchical data processing

**Modern Relevance**: CTEs are fundamental to modern SQL-based data transformation frameworks and are considered essential for any serious data analysis or data engineering work.

I'll continue using this comprehensive format for all future SQL topics!

## Recursive CTEs (Common Table Expressions)

### Brief Explanation

**Recursive CTEs** are specialized Common Table Expressions that reference themselves, enabling queries to process hierarchical or tree-structured data through iterative operations. They consist of two parts: an anchor query (base case) and a recursive query that references the CTE itself.

**Purposes:**
- Navigate hierarchical data structures (organizational charts, category trees)
- Generate sequences and series programmatically
- Traverse graphs and networks
- Calculate cumulative and running totals across levels

**Data Analysis Context:**
- Essential for organizational analytics (reporting chains, team structures)
- Critical for product catalog management (categories, subcategories)
- Fundamental for financial consolidation across business units
- Used in network analysis, social media connections, and supply chain tracing
- Enables complex time-series analysis with dependency relationships

### Examples (Simple to Advanced)

**Simple Example:**
```sql
-- Generate a sequence of numbers
WITH RECURSIVE number_sequence AS (
    -- Anchor: Starting point
    SELECT 1 as num
    
    UNION ALL
    
    -- Recursive: Add next number
    SELECT num + 1
    FROM number_sequence
    WHERE num < 10
)
SELECT num FROM number_sequence;
-- Result: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
```

**Intermediate Example:**
```sql
-- Employee hierarchy navigation
WITH RECURSIVE employee_tree AS (
    -- Anchor: Find all top-level managers (CEO, VPs)
    SELECT 
        employee_id,
        employee_name,
        manager_id,
        job_title,
        1 as level,
        CAST(employee_name AS VARCHAR(1000)) as hierarchy_path
    FROM employees
    WHERE manager_id IS NULL
    
    UNION ALL
    
    -- Recursive: Find all direct reports
    SELECT 
        e.employee_id,
        e.employee_name,
        e.manager_id,
        e.job_title,
        et.level + 1,
        CAST(et.hierarchy_path + ' -> ' + e.employee_name AS VARCHAR(1000))
    FROM employees e
    INNER JOIN employee_tree et ON e.manager_id = et.employee_id
    WHERE et.level < 10  -- Prevent infinite recursion
)
SELECT 
    level,
    REPEAT('  ', level - 1) + employee_name as indented_name,
    job_title,
    hierarchy_path
FROM employee_tree
ORDER BY hierarchy_path;
```

**Advanced Example:**
```sql
-- Complex business analysis: Sales territory hierarchy with aggregations
WITH RECURSIVE territory_hierarchy AS (
    -- Anchor: Root territories (countries/regions)
    SELECT 
        territory_id,
        territory_name,
        parent_territory_id,
        territory_level,
        1 as depth,
        CAST(territory_name AS VARCHAR(500)) as full_path
    FROM territories
    WHERE parent_territory_id IS NULL
    
    UNION ALL
    
    -- Recursive: Child territories
    SELECT 
        t.territory_id,
        t.territory_name,
        t.parent_territory_id,
        t.territory_level,
        th.depth + 1,
        CAST(th.full_path + ' / ' + t.territory_name AS VARCHAR(500))
    FROM territories t
    INNER JOIN territory_hierarchy th ON t.parent_territory_id = th.territory_id
    WHERE th.depth < 5
),
territory_sales AS (
    -- Calculate sales for each territory level
    SELECT 
        th.territory_id,
        th.territory_name,
        th.depth,
        th.full_path,
        COALESCE(SUM(s.sale_amount), 0) as direct_sales,
        COUNT(s.sale_id) as direct_sale_count
    FROM territory_hierarchy th
    LEFT JOIN sales s ON th.territory_id = s.territory_id
    GROUP BY th.territory_id, th.territory_name, th.depth, th.full_path
),
territory_totals AS (
    -- Roll up sales to parent territories
    SELECT 
        ts.territory_id,
        ts.territory_name,
        ts.depth,
        ts.full_path,
        ts.direct_sales,
        ts.direct_sale_count,
        -- Calculate total sales including all child territories
        (SELECT SUM(ts2.direct_sales) 
         FROM territory_sales ts2 
         WHERE ts2.full_path LIKE ts.full_path + '%') as total_sales_with_children
    FROM territory_sales ts
)
SELECT 
    depth,
    REPEAT('  ', depth - 1) + territory_name as hierarchy_display,
    direct_sales,
    total_sales_with_children,
    total_sales_with_children - direct_sales as child_territory_sales,
    ROUND(100.0 * direct_sales / NULLIF(total_sales_with_children, 0), 2) as direct_sales_percentage
FROM territory_totals
ORDER BY full_path;

-- Advanced: Bill of Materials explosion (manufacturing)
WITH RECURSIVE bom_explosion AS (
    -- Anchor: Top-level product
    SELECT 
        product_id,
        component_id,
        quantity_required,
        1 as level,
        quantity_required as total_quantity,
        CAST(product_id + '->' + component_id AS VARCHAR(1000)) as path
    FROM bill_of_materials
    WHERE product_id = 'LAPTOP-001'  -- Starting product
    
    UNION ALL
    
    -- Recursive: Sub-components
    SELECT 
        bom.product_id,
        bom.component_id,
        bom.quantity_required,
        be.level + 1,
        be.total_quantity * bom.quantity_required as total_quantity,
        CAST(be.path + '->' + bom.component_id AS VARCHAR(1000))
    FROM bill_of_materials bom
    INNER JOIN bom_explosion be ON bom.product_id = be.component_id
    WHERE be.level < 10
)
SELECT 
    level,
    REPEAT('  ', level - 1) + component_id as indented_component,
    quantity_required as unit_quantity,
    total_quantity as total_needed,
    path as component_path
FROM bom_explosion
ORDER BY path;
```

### Python and Excel Comparisons

**Python (Pandas + NetworkX):**
```python
import pandas as pd
import networkx as nx

# Recursive hierarchy traversal
def traverse_hierarchy(df, parent_col, child_col, start_node):
    """Simulate recursive CTE for hierarchy traversal"""
    result = []
    
    def recursive_traverse(node, level=1, path=""):
        current_path = f"{path}->{node}" if path else node
        result.append({'node': node, 'level': level, 'path': current_path})
        
        # Find children
        children = df[df[parent_col] == node][child_col].tolist()
        for child in children:
            recursive_traverse(child, level + 1, current_path)
    
    recursive_traverse(start_node)
    return pd.DataFrame(result)

# Using NetworkX for graph traversal
G = nx.from_pandas_edgelist(df, source='manager_id', target='employee_id')
# Get all descendants
descendants = nx.descendants(G, root_node)

# Pandas groupby with cumulative operations (limited recursion simulation)
def hierarchical_sum(df, hierarchy_cols, value_col):
    """Simulate recursive aggregation"""
    result = df.copy()
    for level in range(len(hierarchy_cols)):
        group_cols = hierarchy_cols[:level+1]
        grouped = df.groupby(group_cols)[value_col].sum().reset_index()
        # Additional logic to roll up to parent levels
    return result
```

**Excel:**
- **Hierarchical data**: Use XLOOKUP/INDEX-MATCH with helper columns for level calculation
- **Power Query**: Recursive expansion using Table.ExpandTableColumn
- **Pivot Tables**: Limited hierarchy with drill-down capabilities
- **Manual approach**: Multiple helper columns for each hierarchy level

```excel
# Excel formula approach (limited depth)
=IF(ISBLANK(B2),"",XLOOKUP(B2,A:A,C:C)&"->"&C2)  # Build hierarchy path
=COUNTIF($B$2:B2,B2)  # Calculate level in hierarchy
```

**Power Query M (Excel/Power BI):**
```m
// Recursive function in Power Query
let
    Source = YourTable,
    // Define recursive function
    ExpandHierarchy = (tbl as table, parent as text) =>
        let
            Children = Table.SelectRows(tbl, each [ParentID] = parent),
            AddChildren = Table.AddColumn(Children, "Children", 
                each @ExpandHierarchy(tbl, [ID]))
        in
            AddChildren
in
    ExpandHierarchy(Source, null)
```

### Conclusion and Importance

**Why Recursive CTEs are crucial:**

1. **Unique Problem-Solving Capability**: Recursive CTEs solve problems that are extremely difficult or impossible with standard SQL operations:
   - **Hierarchical queries**: No equivalent in basic SQL
   - **Graph traversal**: Essential for network analysis
   - **Tree operations**: Critical for organizational and product structures

2. **Performance Advantages**:
   - **vs Multiple JOINs**: Eliminates complex self-joins for hierarchical data
   - **vs Application Logic**: Database-level processing is faster than application-level recursion
   - **vs Cursors**: Set-based approach is more efficient than row-by-row processing

3. **Real-World Business Applications**:
   - **Organizational Analytics**: Employee reporting chains, span of control analysis
   - **Financial Consolidation**: Rolling up P&L across business units and subsidiaries
   - **Supply Chain**: Bill of materials explosion, vendor dependency mapping
   - **Customer Analytics**: Referral chains, social network effects

4. **Advantages Over Alternatives**:
   - **vs Stored Procedures with Loops**: Cleaner syntax, better optimization
   - **vs Application-Level Recursion**: Reduced data transfer, leverages database optimization
   - **vs Materialized Views**: Dynamic calculation without storage overhead

5. **Modern Data Architecture Integration**:
   - **Data Warehousing**: Essential for dimensional modeling with hierarchies
   - **Analytics Platforms**: Snowflake, BigQuery, PostgreSQL all optimize recursive CTEs
   - **Graph Databases**: Bridge between relational and graph query paradigms

6. **Advanced Analytical Capabilities**:
   - **Time-series with dependencies**: Calculate cumulative effects across related entities
   - **Network analysis**: Find paths, cycles, and connectivity in business networks
   - **Hierarchical aggregations**: Roll-up reporting across organizational levels

**Best Practices**:
- Always include termination conditions (UNION ALL with WHERE clauses)
- Use depth limits to prevent infinite recursion
- Index foreign key columns used in recursive joins
- Consider materialized views for frequently-accessed hierarchical data
- Test performance with realistic data volumes

**Critical Importance**: Recursive CTEs are irreplaceable for hierarchical data analysis. While Python and Excel offer alternatives, they cannot match SQL's efficiency and elegance for database-resident hierarchical operations. They're essential for any organization dealing with trees, graphs, or multi-level business structures.


# Grouping Sets, Cube, and Rollup in SQL

## What They Mean
**Grouping Sets, Cube, and Rollup** are advanced SQL aggregation techniques that enable **multidimensional analysis** in a single query. They solve the problem of generating multiple levels of summary data without writing complex UNION queries or multiple separate statements.

## Core Purpose
These techniques serve three primary purposes:
- **Data Summarization**: Create hierarchical totals and subtotals across multiple dimensions
- **Business Intelligence**: Generate comprehensive reports showing data at different granular levels
- **Performance Optimization**: Reduce query complexity and execution time compared to multiple separate aggregations

## Data Analysis Significance
In data analysis terms, these are **OLAP (Online Analytical Processing) operations** that enable:
- **Drill-down/Roll-up Analysis**: Moving between different levels of detail
- **Cross-tabulation**: Analyzing data across multiple categorical dimensions
- **Pivot-style Analysis**: Creating summary tables with multiple grouping perspectives
- **Dimensional Analysis**: Understanding data patterns across business hierarchies


## Examples: Simple to Advanced

### Simple Example: Basic Sales Analysis
```sql
-- Simple ROLLUP for time hierarchy
SELECT 
    year, 
    quarter, 
    SUM(sales) as total_sales
FROM sales_data
GROUP BY ROLLUP(year, quarter);
```
**Output includes:**
- Sales by year and quarter
- Sales by year only
- Grand total across all data

### Intermediate Example: Multi-dimensional Analysis
```sql
-- CUBE for product and region analysis
SELECT 
    region,
    product_category,
    COUNT(*) as order_count,
    SUM(revenue) as total_revenue
FROM orders
GROUP BY CUBE(region, product_category);
```
**Output includes all combinations:**
- Region + Product Category
- Region only
- Product Category only  
- Grand total

### Advanced Example: Complex Business Hierarchy
```sql
-- GROUPING SETS for custom business analysis
SELECT 
    fiscal_year,
    quarter,
    region,
    sales_rep,
    product_line,
    SUM(revenue) as revenue,
    COUNT(DISTINCT customer_id) as unique_customers,
    GROUPING(fiscal_year, quarter, region, sales_rep, product_line) as grouping_id
FROM sales_transactions
GROUP BY GROUPING SETS (
    (fiscal_year, quarter, region, sales_rep, product_line),  -- Detailed level
    (fiscal_year, quarter, region),                          -- Regional quarterly
    (fiscal_year, product_line),                             -- Annual product performance
    (sales_rep),                                             -- Individual rep totals
    ()                                                       -- Grand total
);
```


## Python Parallels

### Using Pandas GroupBy with Multiple Levels
```python
import pandas as pd

# Equivalent to SQL ROLLUP
df_rollup = df.groupby(['year', 'quarter']).agg({
    'sales': 'sum'
}).reset_index()

# Add subtotals manually (pandas doesn't have direct ROLLUP)
year_totals = df.groupby('year')['sales'].sum().reset_index()
year_totals['quarter'] = 'Total'
grand_total = pd.DataFrame({
    'year': ['Grand Total'], 
    'quarter': [''], 
    'sales': [df['sales'].sum()]
})

result = pd.concat([df_rollup, year_totals, grand_total])
```

### Using Pandas Pivot Tables (CUBE-like)
```python
# Equivalent to SQL CUBE
pivot_result = pd.pivot_table(
    df, 
    values='revenue',
    index=['region', 'product_category'],
    aggfunc='sum',
    margins=True,  # Adds totals
    fill_value=0
)
```

### Advanced Python: Custom Grouping Sets
```python
def custom_grouping_sets(df, grouping_sets, agg_dict):
    results = []
    for grouping in grouping_sets:
        if grouping:  # Not empty tuple
            grouped = df.groupby(list(grouping)).agg(agg_dict).reset_index()
        else:  # Grand total
            grouped = df.agg(agg_dict).to_frame().T
        results.append(grouped)
    return pd.concat(results, ignore_index=True)
```


## Excel Parallels

### Pivot Tables (CUBE equivalent)
```
Excel Pivot Table:
- Rows: Region, Product Category
- Values: Sum of Revenue, Count of Orders
- Show Grand Totals: Yes
- Show Subtotals: Yes
```

### Subtotal Function (ROLLUP equivalent)
```
Excel Formula approach:
=SUBTOTAL(9, range)  # For sums at different hierarchy levels
```

### Power Query (Advanced Grouping Sets)
```
Power Query M:
Table.Group(
    Source,
    {"Year", "Quarter"},
    {{"Total Sales", each List.Sum([Sales]), type number}}
)
```


## Importance vs Alternative Methods

### Why These Techniques Excel Over Alternatives

**1. Performance Superiority**
- **Single Query Execution**: One pass through data vs multiple queries
- **Optimized Execution Plans**: Database engines optimize these operations specifically
- **Reduced I/O**: Eliminates need for multiple table scans

**2. Code Simplicity vs UNION Approach**
```sql
-- Traditional approach (inefficient)
SELECT dept, NULL as job, SUM(salary) FROM emp GROUP BY dept
UNION ALL
SELECT NULL, job, SUM(salary) FROM emp GROUP BY job
UNION ALL
SELECT NULL, NULL, SUM(salary) FROM emp;

-- Modern approach (efficient)  
SELECT dept, job, SUM(salary) 
FROM emp 
GROUP BY GROUPING SETS ((dept), (job), ());
```

**3. Business Intelligence Integration**
- **OLAP Compatibility**: Direct integration with BI tools like Tableau, Power BI
- **Dimensional Modeling**: Natural fit for star/snowflake schemas
- **Report Generation**: Single query produces complete analytical reports

**4. Analytical Flexibility**
- **Dynamic Hierarchies**: Easy to modify grouping levels without query restructuring  
- **Cross-dimensional Analysis**: Analyze relationships between different business dimensions
- **Scalable Complexity**: Handles both simple subtotals and complex multidimensional analysis

### When Alternatives Fall Short
- **Manual UNION queries**: Become exponentially complex with more dimensions
- **Application-level aggregation**: Performance bottlenecks with large datasets
- **Multiple separate queries**: Network overhead and consistency issues
- **Spreadsheet pivot tables**: Limited scalability and processing power


---

# STRING_AGG and Array Aggregations in SQL

## What They Mean
**STRING_AGG** and **Array Aggregations** are SQL functions that combine multiple values from grouped rows into a single aggregated result. STRING_AGG creates concatenated strings, while array aggregations (like ARRAY_AGG) create arrays/lists of values. These represent **value consolidation techniques** that transform normalized relational data into denormalized, collection-based formats.

## Core Purpose
These functions serve three primary purposes:
- **Data Denormalization**: Convert multiple related rows into single consolidated records
- **Reporting Enhancement**: Create readable, comma-separated lists for business reports
- **Data Export Preparation**: Format data for downstream systems that expect consolidated formats

## Data Analysis Significance
In data analysis terms, these are **aggregation transformations** that enable:
- **One-to-Many Relationship Flattening**: Collapse child records into parent summaries
- **Categorical Data Consolidation**: Combine multiple categories into single descriptive fields
- **ETL Processing**: Prepare data for systems expecting consolidated formats
- **Analytical Reporting**: Create human-readable summaries from normalized data structures



## Examples: Simple to Advanced

### Simple Example: Basic Customer Order Summary
```sql
-- STRING_AGG: Create comma-separated list of products ordered
SELECT 
    customer_id,
    customer_name,
    STRING_AGG(product_name, ', ') as products_ordered,
    COUNT(*) as total_orders
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN products p ON o.product_id = p.product_id
GROUP BY customer_id, customer_name;

-- ARRAY_AGG: Create array of order amounts
SELECT 
    customer_id,
    ARRAY_AGG(order_amount) as order_amounts,
    ARRAY_AGG(DISTINCT product_category) as categories_purchased
FROM orders o
JOIN products p ON o.product_id = p.product_id
GROUP BY customer_id;
```

### Intermediate Example: Enhanced Reporting with Ordering
```sql
-- STRING_AGG with custom separators and ordering
SELECT 
    department,
    STRING_AGG(
        employee_name || ' ($' || salary || ')', 
        ' | ' 
        ORDER BY salary DESC
    ) as employee_salary_list,
    ARRAY_AGG(
        DISTINCT skill 
        ORDER BY skill
    ) as department_skills
FROM employees e
LEFT JOIN employee_skills es ON e.employee_id = es.employee_id
GROUP BY department;

-- Advanced formatting with conditional aggregation
SELECT 
    project_id,
    project_name,
    STRING_AGG(
        CASE 
            WHEN role = 'Lead' THEN '**' || employee_name || '**'
            ELSE employee_name 
        END,
        ', '
        ORDER BY 
            CASE WHEN role = 'Lead' THEN 1 ELSE 2 END,
            employee_name
    ) as team_members,
    ARRAY_AGG(DISTINCT technology ORDER BY technology) as tech_stack
FROM projects p
JOIN project_assignments pa ON p.project_id = pa.project_id
JOIN employees e ON pa.employee_id = e.employee_id
GROUP BY project_id, project_name;
```

### Advanced Example: Complex Business Intelligence
```sql
-- Multi-level aggregation with JSON-like formatting
WITH sales_summary AS (
    SELECT 
        s.sales_rep_id,
        sr.rep_name,
        EXTRACT(YEAR FROM s.sale_date) as year,
        EXTRACT(QUARTER FROM s.sale_date) as quarter,
        c.industry,
        STRING_AGG(
            DISTINCT c.company_name 
            ORDER BY c.company_name
        ) as clients,
        ARRAY_AGG(
            JSON_BUILD_OBJECT(
                'deal_id', s.deal_id,
                'amount', s.deal_amount,
                'product', p.product_name,
                'close_date', s.sale_date
            ) ORDER BY s.deal_amount DESC
        ) as deals_detail,
        STRING_AGG(
            DISTINCT p.product_category || ' (' || COUNT(s.deal_id) || ')',
            ', '
        ) as product_performance
    FROM sales s
    JOIN sales_reps sr ON s.sales_rep_id = sr.rep_id
    JOIN customers c ON s.customer_id = c.customer_id
    JOIN products p ON s.product_id = p.product_id
    WHERE s.sale_date >= CURRENT_DATE - INTERVAL '2 years'
    GROUP BY s.sales_rep_id, sr.rep_name, year, quarter, c.industry
),
performance_metrics AS (
    SELECT 
        sales_rep_id,
        rep_name,
        STRING_AGG(
            DISTINCT industry || ': ' || clients,
            '; '
            ORDER BY industry
        ) as industry_client_map,
        ARRAY_AGG(
            DISTINCT ARRAY[year::text, quarter::text, deals_detail::text]
        ) as quarterly_performance
    FROM sales_summary
    GROUP BY sales_rep_id, rep_name
)
SELECT 
    rep_name,
    industry_client_map,
    ARRAY_LENGTH(quarterly_performance, 1) as active_quarters
FROM performance_metrics
ORDER BY rep_name;
```



## Python Parallels

### Basic String and List Aggregation
```python
import pandas as pd

# STRING_AGG equivalent using pandas
df_string_agg = df.groupby('customer_id').agg({
    'product_name': lambda x: ', '.join(x.unique()),  # STRING_AGG equivalent
    'customer_name': 'first',  # Get customer name
    'order_id': 'count'  # Count orders
}).reset_index()

# ARRAY_AGG equivalent using pandas
df_array_agg = df.groupby('customer_id').agg({
    'order_amount': list,  # ARRAY_AGG equivalent
    'product_category': lambda x: list(x.unique())  # ARRAY_AGG DISTINCT
}).reset_index()
```

### Advanced Pandas Aggregations
```python
# Complex aggregation with custom formatting
def format_employee_list(group):
    """Custom function to format employee salary list"""
    sorted_employees = group.sort_values('salary', ascending=False)
    return ' | '.join([
        f"{'**' if row.role == 'Lead' else ''}{row.employee_name}{'**' if row.role == 'Lead' else ''} (${row.salary})"
        for _, row in sorted_employees.iterrows()
    ])

# Apply custom aggregation
result = df.groupby('department').agg({
    'employee_name': format_employee_list,
    'skill': lambda x: sorted(x.unique()),
    'salary': ['mean', 'count']
}).reset_index()

# Using transform for more complex operations
df['product_list'] = df.groupby('customer_id')['product_name'].transform(
    lambda x: ', '.join(x.unique())
)
```

### JSON-like Aggregation in Python
```python
import json

# Complex nested aggregation
def create_deals_summary(group):
    deals = []
    for _, row in group.iterrows():
        deal = {
            'deal_id': row['deal_id'],
            'amount': row['deal_amount'],
            'product': row['product_name'],
            'close_date': row['sale_date'].strftime('%Y-%m-%d')
        }
        deals.append(deal)
    return sorted(deals, key=lambda x: x['amount'], reverse=True)

# Apply complex aggregation
sales_summary = df.groupby(['sales_rep_id', 'year', 'quarter']).agg({
    'company_name': lambda x: ', '.join(x.unique()),
    'deal_id': create_deals_summary,
    'product_category': lambda x: list(x.unique())
}).reset_index()
```



## Excel Parallels

### Basic Text Concatenation
```excel
# Excel TEXTJOIN function (STRING_AGG equivalent)
=TEXTJOIN(", ", TRUE, IF(A:A=customer_id, B:B, ""))

# Excel array formula for unique values
=TEXTJOIN(", ", TRUE, UNIQUE(IF(A:A=customer_id, B:B, "")))
```

### Power Query Aggregations
```
Power Query M Language:
// STRING_AGG equivalent
= Table.Group(
    Source,
    {"customer_id"},
    {{"products", each Text.Combine([product_name], ", "), type text}}
)

// ARRAY_AGG equivalent  
= Table.Group(
    Source,
    {"customer_id"}, 
    {{"order_amounts", each [order_amount], type list}}
)
```

### Advanced Excel Techniques
```excel
# Conditional concatenation
=TEXTJOIN(", ", TRUE, 
    IF((A:A=customer_id)*(C:C="Premium"), 
       B:B & " (Premium)", 
       IF(A:A=customer_id, B:B, "")))

# Using FILTER with TEXTJOIN (Excel 365)
=TEXTJOIN(", ", TRUE, 
    UNIQUE(FILTER(product_name, customer_id_range=target_customer)))
```



## Importance vs Alternative Methods

### Why STRING_AGG and Array Aggregations Excel

**1. Performance Advantages**
```sql
-- Efficient single-query approach
SELECT 
    customer_id,
    STRING_AGG(product_name, ', ') as products
FROM orders o
JOIN products p ON o.product_id = p.product_id
GROUP BY customer_id;

-- vs. Inefficient application-level concatenation requiring multiple queries
-- Query 1: Get customers
-- Query 2: For each customer, get products
-- Application: Concatenate strings
```

**2. Database-Level Optimization**
- **Memory Efficiency**: Database engines optimize string concatenation
- **Parallel Processing**: Can leverage database parallelization
- **Index Usage**: GROUP BY operations can use indexes effectively

**3. Data Consistency**
- **Atomic Operations**: Ensures data consistency within single transaction
- **Concurrent Access**: Handles multiple users accessing same data
- **Snapshot Isolation**: Provides consistent view of data during aggregation

### Comparison with Alternative Approaches

**1. Application-Level Concatenation**
```python
# Inefficient approach
customers = []
for customer_id in get_customer_ids():
    products = get_products_for_customer(customer_id)  # Multiple DB calls
    product_list = ', '.join([p.name for p in products])
    customers.append({'id': customer_id, 'products': product_list})

# vs. Efficient single query with STRING_AGG
# One database call, processed efficiently by database engine
```

**2. Multiple Queries with UNION**
```sql
-- Cumbersome alternative
SELECT customer_id, GROUP_CONCAT(...) FROM orders WHERE customer_id = 1
UNION ALL
SELECT customer_id, GROUP_CONCAT(...) FROM orders WHERE customer_id = 2
-- ... continues for each customer

-- vs. Clean aggregation approach
SELECT customer_id, STRING_AGG(product_name, ', ')
FROM orders GROUP BY customer_id;
```

**3. Cursor-Based Processing**
```sql
-- Old-school approach (inefficient)
DECLARE cursor_name CURSOR FOR SELECT DISTINCT customer_id FROM orders;
-- Loop through each customer
-- Build string manually
-- Much slower and more complex

-- vs. Set-based aggregation (efficient)
SELECT customer_id, STRING_AGG(product_name, ', ')
FROM orders GROUP BY customer_id;
```

### Advanced Use Cases and Benefits

**1. Reporting and Business Intelligence**
- **Executive Dashboards**: Create readable summaries for business stakeholders
- **Data Export**: Prepare data for Excel/CSV exports with consolidated formats
- **Email Reporting**: Generate formatted lists for automated business communications

**2. ETL and Data Integration**
- **Data Warehouse Loading**: Consolidate operational data for analytical systems
- **API Responses**: Format data for REST APIs expecting consolidated responses
- **Migration Scripts**: Prepare data for systems with different schema requirements

**3. Performance Optimization Strategies**
```sql
-- Optimize with selective aggregation
SELECT 
    category,
    STRING_AGG(
        product_name, 
        ', ' 
        ORDER BY sales_rank 
        LIMIT 10  -- Only top 10 products per category
    ) as top_products
FROM products
WHERE is_active = true
GROUP BY category;
```

**4. Advanced Formatting Techniques**
```sql
-- Custom formatting with business logic
SELECT 
    department,
    STRING_AGG(
        CASE 
            WHEN performance_rating >= 4.5 THEN '⭐ ' || employee_name
            WHEN performance_rating >= 4.0 THEN '✓ ' || employee_name  
            ELSE employee_name
        END,
        ', '
        ORDER BY performance_rating DESC, employee_name
    ) as team_roster
FROM employees
GROUP BY department;
```


## Practice Questions

1. Write a query to find the 2nd highest salary from an employee table.

    ```sql
    SELECT
    SALARY
    FROM 
    EMPLOYEES
    ORDER BY SALARY
    LIMIT 1
    OFFSET 1
    ```

2. Write a SQL query to find employees who have the same manager.

3. Write a query to find the first and last record for each employee based on the 'hire_date' column.

4. Write a query to find the most recent transaction for each customer.

5. Write a query to find the total salary of each department and display departments with a total salary greater than a specified value (e.g., 50,000).

6. Write a query to find the running total of orders for each customer sorted by order date.

7. Write a query to get the total number of employees hired per month and year.

8. Write a query to display all employees who earn more than the average salary for their department.

9.  Write a query to get the second-lowest salary from the employee table without using LIMIT or OFFSET.

10. Write a query to list all products that have never been ordered (assuming an 'orders' table and a 'products' table).

11. Write a query to list all the employees who are also managers.

12. Write a query to find employees who have joined in the same month and year.

13. Write a SQL query to get a list of employees who are older than the average age of all employees in their department.

14. Write a query to find the employee(s) with the longest tenure at the company.

15. Write a query to delete all records from a table where the column value is NULL.

16. Write a query to find all pairs of products that were ordered together at least once.

17. Write a query to find the average order value by customer for each month.

18. Write a query to identify customers who made a purchase every month for the past year.

19. Write a query to find the total revenue for each product in a given quarter.

20. Write a query to find the first purchase date of each customer.

21. Write a query to calculate the year-on-year growth of revenue.

22. Write a query to find the 'Nth' highest salary from the employee table (e.g., 5th highest salary).